# Libraries

Import libraries needed in this project.

In [99]:
import warnings
warnings.filterwarnings('ignore')
import cv2  #For image and video processing and visualization
import os   #To interact with operating system and files
import numpy as np  #For matrix operations
import random   #To generate random numbers
from sklearn.mixture import GaussianMixture  #For clustering
from sklearn.cluster import DBSCAN   #For clustering
from sklearn.cluster import MeanShift, estimate_bandwidth  #For clustering
import numba
from PIL import Image
from scipy.ndimage import gaussian_filter
from skimage.metrics import structural_similarity as ssim
from scipy.stats import entropy
import torch
import torch.nn.functional as Fun
from torchvision import transforms
from torch import optim
import time
from torch import nn
from torchvision import transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from scipy.interpolate import splprep, splev

# Open Video

This function recieves a video path and returns a capture stream.

In [3]:
def open_vid(input_file): #Video path
    cap = cv2.VideoCapture(input_file) #Open capture stream
    if not cap.isOpened(): #Check if is available
        print("Error: Could not open video.")
    return cap

# Get Video Properties

This function gets the video's properties for its width and height in pixels, frames per second (fps) and frame count.

In [4]:
def get_props(cap, display=1): #Video capture stream and flag to display properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) #Get Width
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) #Get Height
    fps = cap.get(cv2.CAP_PROP_FPS) #Get FPS
    count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) #Get Frame Count
    if display==1:  #If flag is 1, display properties
        print("Width: ",width)
        print("Height: ",height)
        print("FPS: ",fps)
        print("Frame Count: ",count)
    return width,height,fps,count

# Get Frames from Video as a List 

This function takes the capture stream of a video and saves its frames in a list.

In [5]:
def get_frames(cap): #Video capture stream
    frames = [] #Frames list
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) #Get Frame count
    for i in range(frame_count): #For each frame
        ret, frame = cap.read() # Read a frame from the video
        if not ret: #If couldn't read frame
            print("Error: Could not read frame.") #Display error message and return read frames
            return frames
        frames.append(frame) # Save the frame to the list
    if not frames: #If list is empty
        print("No frames were saved.") #Display error message 
    return frames

# Delete PNG Files in Directory

This function deletes all PNG image files in the specified directory path.

In [6]:
def delete_png_files(directory_path): #Directory path to delete all PNG files
    for filename in os.listdir(directory_path): # List all files in the specified directory
        file_path = os.path.join(directory_path, filename) # Construct full file path
        try:
            if os.path.isfile(file_path) and filename.lower().endswith('.png'): # Check if it's a PNG file and remove it
                os.remove(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')  #If not, display message

# Save Frames as Images

This function saves a list of frames as images in the given directory path.

In [7]:
def save_frames(frames, frame_folder): #Frame list, directory path to be saved
    if not os.path.exists(frame_folder):  # Create the folder to save frames if it doesn't exist
        os.makedirs(frame_folder)
    frame_count = 0 #frame index
    for i in range(len(frames)): #for each frame
        frame_filename = os.path.join(frame_folder, f'frame_{frame_count:03d}.png') #directory path and file name
        cv2.imwrite(frame_filename, frames[i]) # Save the frame as an image file
        frame_count += 1 #Next frame index

# Read Images in a Directory

This function reads and stores images from a given directory path to a list.

In [8]:
def read_images(directory_path):
    images = []
    # List all files in the directory
    for filename in os.listdir(directory_path):
        # Check if the file has a PNG extension
        if filename.lower().endswith('.png'):
            # Construct full file path
            file_path = os.path.join(directory_path, filename)
            # Read the image using OpenCV
            image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)  # cv2.IMREAD_UNCHANGED to keep the alpha channel if present
            if image is not None:
                images.append(image)
            else:
                print(f"Failed to read image: {file_path}")
    return images

# Create Video with List of Frames

This function creates and saves the frames in a list into a video file in the specified directory path.

In [9]:
def save_vid(frames,output_file,fps): #List of frames, directory path to be saved, fps
    height, width, _ = frames[0].shape     # Get frame dimensions
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Define the codec and create VideoWriter object
    out = cv2.VideoWriter(output_file, fourcc, fps, (width, height)) 
    [out.write(frame) for frame in frames]; # Write the frames to the new video file
    out.release() # Release the video writer object

# Display Image

This function displays a given image in a window.

In [10]:
def display_frame(I): #Image to display
    cv2.imshow('I', I)  #Display Image
    cv2.waitKey(0) #Press any key to stop displaying
    cv2.destroyAllWindows() #Close all windows

This function display a list of images at the same time.

In [11]:
def display_images(images):
    for i, img in enumerate(images):
        window_name = f'Image {i+1}'
        cv2.imshow(window_name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Play Frames in a Window

This function displays a video made out of a list of frames with the specified FPS.

In [12]:
def play_frames(frames,fps): #list of frames, fps
    delay = int(1000/fps) #Delay between frames
    print("Delay: ",delay)
    for frame in frames:
        cv2.imshow('Video Playback', frame) # Display the frame
        if cv2.waitKey(delay) & 0xFF == ord('q'): # Exit the playback if 'q' is pressed
            break
    cv2.destroyAllWindows() #Close all windows

## Display Frames in List

In [ ]:
def ThroughFrames(frames):
    i = 0
    while True:
        cv2.imshow('Frame', frames[i])
         # Wait for a key press to move to the next frame
        key = cv2.waitKeyEx(0)
        if key == ord('q'):
            break
        if key == 2424832:  # Left arrow key
            i = i - 1
            if i<0:
                i = 0
        if key == 2555904:  # Right arrow key
            i = i + 1
            if i>(len(frames)-1):
                i = len(frames)-1
    # Release the video capture object
    cv2.destroyAllWindows()

# Optical Flow

Optical Flow can be seen as a vector field that describes the movement between two consecutive images or frames in a video. There are many ways to calculate the oprical flow. Some of the methods to solve optical flow are:
- Ferneback
- Lucas - Kanade
- Phase Correlation

## Farneback Magnitude

This function take a list of frames to return a list of the optical flow's vector fields and display the magnitude of these vectors.

In [13]:
def OFM(frames): #List of frames
    OF = []  #List of optical flow's vector field
    prev_gray = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY) #Convert to Grayscale
    i = 0  #frame's index
    while True:
        print(i,end='\r') #Print frame index
        next_gray=cv2.cvtColor(frames[(i+1)%len(frames)],cv2.COLOR_BGR2GRAY)#Convert current & next frame to grayscale
        # Calculate the dense optical flow using Farneback method
        flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, 0.5, 3, 10, 10, 5, 1.2, 0)
                                            # prev     next      flow  dist lvl win it smooth std  flag
        # Visualize the optical flow
        hsv = np.zeros_like(frames[i]) #Matrix with shape like frames with zeros
        if len(hsv.shape) != 3 or hsv.shape[2] != 3: #If color image
            hsv = np.zeros((frames[i].shape[0], frames[i].shape[1], 3)) #Matrix of size of frame with 3 color channels
        hsv[..., 1] = 255 # ch1 Saturation (Full)
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1]) # Cartesian to Polar
        hsv[..., 0] = ang * 180 / np.pi / 2  #Angle
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX) #Normalize from 0 to 255
        flow_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)   #Convert to RGB
        OF.append(flow) #Add to list of Optical Flow
        flow_M  = flow[...,0] + flow[...,1] #Add components of vectors
        flow_M = flow_M/np.max(flow_M)*255 #Calculate Magnitude of vectors     
        # Display the original frames and the optical flow magnitudes
        cv2.imshow('Prev Frame', frames[i])
        cv2.imshow('Next Frame', frames[(i+1) % len(frames)])
        cv2.imshow('Optical Flow', flow_rgb)
        cv2.imshow('Optical Flow Mag', flow_M)
        # Wait for a key press to move to the next frame
        key = cv2.waitKeyEx(0)
        if key == ord('q'):
            break
        if key == 2424832:  # Left arrow key to move backwards
            i = i - 1
            if i<0:
                i = 0
        if key == 2555904:  # Right arrow key to move forwards
            i = i + 1
            if i>(len(frames)-1):
                i = len(frames)-1
        prev_gray = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY) # Update the previous frame and grayscale image
    cv2.destroyAllWindows() # Release the video capture object and close all OpenCV windows
    return OF

## Draw Optical Flow Vector Field and Sum of all Vectors

These functions recieve the optical flow and the frame to draw onto.

In [14]:
def draw_optical_flow_vectors(flow, frame, step): #Optical flow, frame, window size
    h, w = frame.shape[:2] #frame size
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2, -1).astype(int) #Grid of window centers
    fx, fy = flow[y, x].T #Separate flow components
    mask = np.zeros_like(frame) # Create a mask to draw the vectors  
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2) # Create line endpoints
    lines = np.int32(lines + 0.5) # Add space between
    # Draw lines and circles for each vector
    for (x1, y1), (x2, y2) in lines:
        cv2.line(mask, (x2, y2), (x1, y1), (0, 255, 0), 1)
        cv2.circle(frame, (x2, y2), 1, (0, 255, 0), -1)
    return cv2.add(frame, mask) #Draw mask on top of frame

def draw_sum_vector(flow, frame):
    h, w = frame.shape[:2] #Frame size
    # Compute the sum of all flow vectors by components
    sum_fx = np.sum(flow[..., 0])
    sum_fy = np.sum(flow[..., 1])
    center_x, center_y = w // 2, h // 2 # Calculate the center point of the frame
    # Normalize the sum vector to fit within the image
    max_length = min(w, h) // 2
    vector_length = np.sqrt(sum_fx**2 + sum_fy**2)
    if vector_length > 0:
        scale = max_length / vector_length #scale factor
        end_x = int(center_x + sum_fx * scale)
        end_y = int(center_y + sum_fy * scale)
    else:
        end_x, end_y = center_x, center_y
    # Draw the sum vector as a red arrow
    frame_with_vector = np.copy(frame) #copy original frame
    cv2.arrowedLine(frame_with_vector, (center_x, center_y), (end_x, end_y), (0, 0, 255), 2, tipLength=0.2) 
    return frame_with_vector

## Farneack with Vectors

The function OFV take a list of frames to return and display the optical flow's vector fields as well as the sum of all vectors.

In [15]:
def OFV(frames, step=15): #Frame list, window size
    OF = [] #List of Optical flow's vector fields
    i = 0 #Frame's index
    while True:
        #convert it to grayscale
        prev_gray = cv2.cvtColor(np.copy(frames[i]), cv2.COLOR_BGR2GRAY)
        next_gray = cv2.cvtColor(np.copy(frames[i+1]), cv2.COLOR_BGR2GRAY)
        # Calculate the dense optical flow using Farneback method
        flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        # Draw the optical flow vectors on the frame
        flow_frame = draw_optical_flow_vectors(flow, cv2.addWeighted(frames[i], 0.5, frames[i+1], 0.5, 0), step)
        OF.append(flow) #Add flow to list
        # Draw the sum vector on the frame
        frame_with_sum_vector = draw_sum_vector(flow, flow_frame)
        # Display the original frame with optical flow vectors and sum vector
        cv2.imshow('Prev Frame', frames[i])
        cv2.imshow('Next Frame', frames[i+1])
        #cv2.imshow('Optical Flow Vectors', flow_frame)
        cv2.imshow('Sum Vector', frame_with_sum_vector)
        key = cv2.waitKeyEx(0) #Read pressed key
        if key == ord('q'): #Stop if pressed
            break
        if key == 2424832:  # Left arrow key for previous frames
            i = i - 1
            if i < 0: #prevent non existent frames
                i = 0
        if key == 2555904:  # Right arrow key for next frames
            i = i + 1
            if i > (len(frames) - 2): #prevent non existent frames
                i = len(frames) - 2
    cv2.destroyAllWindows() #Close all windows
    return OF

## Lucas-Kanade

This function utilizes the Lukas-Kanade method to solve and display the optical flow.

In [16]:
def OFLK(frames, step_size=15): #Video Frames, window size
    OF = [] #List of optical flow
    lk_params=dict(winSize=(step_size,step_size),maxLevel=10,criteria=(cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT,30,0.01))
    #lucas-kanade parameters  #window Size        #PyramidLevel   #End Criteria
    i = 0 #Frame index
    while i < len(frames) - 1:
        #Convert to Grayscale
        old_gray = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY) 
        frame_gray = cv2.cvtColor(frames[i + 1], cv2.COLOR_BGR2GRAY)
        #Neighborhood grid centers
        grid_y, grid_x = np.mgrid[step_size//2:old_gray.shape[0]:step_size, step_size//2:old_gray.shape[1]:step_size]
        p0 = np.vstack((grid_x.ravel(), grid_y.ravel())).T.astype(np.float32).reshape(-1, 1, 2)
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params) #Calculate Optical Flow
        #Set to 1 if change has been found
        good_new = p1[st == 1]
        good_old = p0[st == 1]
        #Frames average
        frame = cv2.addWeighted(frames[i], 0.5, frames[i + 1], 0.5, 0)
        #Calculate Vector Field
        flow = np.zeros((old_gray.shape[0], old_gray.shape[1], 2))
        flow[good_old[:, 1].astype(int), good_old[:, 0].astype(int), 0] = good_new[:, 0] - good_old[:, 0]
        flow[good_old[:, 1].astype(int), good_old[:, 0].astype(int), 1] = good_new[:, 1] - good_old[:, 1]
        #Draw Vectors
        frame_with_vectors = draw_optical_flow_vectors(flow, frame, step_size)
        frame_with_sum_vector = draw_sum_vector(flow, frame_with_vectors)
        OF.append(flow) #add to flow list
        #Display Frames and Flow
        cv2.imshow('Previous Frame',frames[i])
        cv2.imshow('Next Frame', frames[i+1])
        #cv2.imshow('Optical Flow Vectors', frame_with_vectors)
        cv2.imshow('Sum Vector', frame_with_sum_vector)
        key = cv2.waitKeyEx(0)
        if key == ord('q'):
            break
        if key == 2424832:  # Left arrow key to move backward
            i = max(0, i - 1)
        if key == 2555904:  # Right arrow key to move forward
            i = min(len(frames) - 2, i + 1)
    cv2.destroyAllWindows()
    return OF

## Phase Correlation

The PhaseC function calculates and displays the optical flow of the given frames by using Phase Correlation.

In [17]:
def draw_sum_vector_phase_c(sum_dx, sum_dy, frame): #Y component, X component, frame to drawn on
    h, w = frame.shape[:2] #Size
    center_x, center_y = w // 2, h // 2 # Calculate the center point of the frame
    # Normalize the sum vector to fit within the image
    max_length = min(w, h) // 2
    vector_length = np.sqrt(sum_dx**2 + sum_dy**2)
    if vector_length > 0:
        scale = 0.1#max_length / vector_length #Scale factor
        end_x = int(center_x + sum_dx * scale)
        end_y = int(center_y + sum_dy * scale)
    else:
        end_x, end_y = center_x, center_y
    # Draw the sum vector as a red arrow
    frame_with_vector = np.copy(frame)
    cv2.arrowedLine(frame_with_vector, (center_x, center_y), (end_x, end_y), (0, 0, 255), 2, tipLength=0.2)
    return frame_with_vector

def PhaseC(frames, block_size=15, grid_step=15):
    OF = [] #Optical Flow List
    i = 0 #Frame index
    while True:
        prev_frame = np.copy(frames[i])
        next_frame = np.copy(frames[i + 1])

        # Convert frames to grayscale
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)

        # Create an image to visualize the flow
        flow_img = cv2.cvtColor(prev_gray, cv2.COLOR_GRAY2BGR)
        frame = cv2.addWeighted(frames[i], 0.5, frames[i+1], 0.5, 0)

        sum_dx, sum_dy = 0, 0

        # Iterate over the grid
        for y in range(0, prev_gray.shape[0] - block_size, grid_step):
            for x in range(0, prev_gray.shape[1] - block_size, grid_step):
                # Extract the blocks
                prev_block = prev_gray[y:y + block_size, x:x + block_size]
                next_block = next_gray[y:y + block_size, x:x + block_size]

                # Compute phase correlation
                shift, _ = cv2.phaseCorrelate(prev_block.astype(np.float32), next_block.astype(np.float32))
                dx, dy = shift

                # Sum the vectors
                sum_dx += dx
                sum_dy += dy

                # Scale down the length of the arrows and size of the tips
                scale = 1
                tip_length = 0.2

                # Draw the vector on the flow image
                cv2.arrowedLine(frame, (x + block_size // 2, y + block_size // 2),
                                (int(x + block_size // 2 + dx * scale), int(y + block_size // 2 + dy * scale)),
                                (0, 255, 0), 1, tipLength=tip_length)

        # Draw the sum vector on the frame
        frame_with_sum_vector = draw_sum_vector_phase_c(sum_dx, sum_dy, frame)

        OF.append(flow_img)

        # Display the frames and the flow
        cv2.imshow('Previous Frame', prev_frame)
        cv2.imshow('Next Frame', next_frame)
        #cv2.imshow('Optical Flow', frame)
        cv2.imshow('Sum Vector', frame_with_sum_vector)

        # Wait for a key press to move to the next frame
        key = cv2.waitKeyEx(0)
        if key == ord('q'):
            break
        if key == 2424832:  # Left arrow key
            i = i - 1
            if i < 0:
                i = 0
        if key == 2555904:  # Right arrow key
            i = i + 1
            if i > (len(frames) - 2):
                i = len(frames) - 2

    cv2.destroyAllWindows()
    return OF

# Real Time Optical Flow with Camera

This function captures video from a camera and displays it with the optical flow using Furneback's method.

In [18]:
def draw_optical_flow(frame, flow, step=16, max_magnitude=100):
    h, w = frame.shape[:2]
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    lines = np.vstack([x, y, x + fx, y + fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    vis = frame.copy()
    for (x1, y1), (x2, y2) in lines:
        cv2.arrowedLine(vis, (x1, y1), (x2, y2), (0, 255, 0), 1, tipLength=0.4)

    # Compute the sum of all flow vectors
    sum_fx = np.sum(fx)
    sum_fy = np.sum(fy)
    
    # Normalize the sum vector
    norm = np.sqrt(sum_fx**2 + sum_fy**2)
    if norm > 0:
        sum_fx /= norm
        sum_fy /= norm

    # Scale the normalized vector by the maximum magnitude
    sum_fx *= min(norm, max_magnitude)
    sum_fy *= min(norm, max_magnitude)
    
    # Draw the red vector representing the normalized sum of all flow vectors
    center_x, center_y = w // 2, h // 2
    end_x = int(center_x + sum_fx)  # Scale for better visualization
    end_y = int(center_y + sum_fy)
    cv2.arrowedLine(vis, (center_x, center_y), (end_x, end_y), (0, 0, 255), 2, tipLength=0.4)
    
    return vis

def cap_of():
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    ret, prev_frame = cap.read()
    if not ret:
        print("Error: Could not read initial frame.")
        return
    
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 
                                            0.5, 3, 15, 3, 5, 1.2, 0)
        
        prev_gray = gray

        # Draw the optical flow vectors on the RGB frame
        optical_flow_frame = draw_optical_flow(frame, flow)
        
        # Display the resulting frame with optical flow
        cv2.imshow('Webcam Video with Optical Flow', optical_flow_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Frames Differences

This function takes a list of frames of a video and calculates the absolute diffences between two consecutive frames with a threshold to display a binary image with the regions where the most differences are.

In [19]:
def frame_dif(frames, threshold=30): #List of frames, threshold
    D = [] #Differences List
    i = 0 #Frame index
    while True:
        # Convert frames to grayscale
        prev_gray = cv2.cvtColor(np.copy(frames[i]),cv2.COLOR_BGR2GRAY)
        curr_gray = cv2.cvtColor(np.copy(frames[i+1]), cv2.COLOR_BGR2GRAY)
        # Compute absolute difference between frames
        diff = cv2.absdiff(prev_gray, curr_gray) 
        # Create a mask to highlight pixels with significant changes
        mask = np.zeros_like(diff)
        mask[diff > threshold] = 255 #Apply Threshold
        D.append(mask) #Add to list
        # Show original frame and mask
        cv2.imshow('Prev Frame', frames[i])
        cv2.imshow('Next Frame', frames[i+1])
        cv2.imshow('Pixels with Most Changes', mask)
         # Wait for a key press to move to the next frame
        key = cv2.waitKeyEx(0)
        if key == ord('q'):
            break
        if key == 2424832:  # Left arrow key
            i = i - 1
            if i<0:
                i = 0
        if key == 2555904:  # Right arrow key
            i = i + 1
            if i>(len(frames)-2):
                i = len(frames)-2
        prev_frame = frames[i]
    # Release the video capture object
    cv2.destroyAllWindows()
    return D

This function takes a list of frames and calculates the normalized absolute difference between 2 consecutive frames.

In [20]:
def frame_dif1(frames): #List of frames, threshold
    D = [] #Differences List
    i = 0 #Frame index
    while True:
        # Convert frames to grayscale
        prev_gray = cv2.cvtColor(np.copy(frames[i]), cv2.COLOR_BGR2GRAY)
        curr_gray = cv2.cvtColor(np.copy(frames[i+1]), cv2.COLOR_BGR2GRAY)
        # Compute absolute difference between frames
        diff = abs(prev_gray-curr_gray) #Absolute difference
        dif = (1/diff.max())*diff*255 #Normalize
        D.append(diff) #Add to list
        # Show original frame and mask
        cv2.imshow('Prev Frame', frames[i])
        cv2.imshow('Next Frame', frames[i+1])
        cv2.imshow('Pixels with Most Changes', diff)
         # Wait for a key press to move to the next frame
        key = cv2.waitKeyEx(0)
        if key == ord('q'):
            break
        if key == 2424832:  # Left arrow key
            i = i - 1
            if i<0:
                i = 0
        if key == 2555904:  # Right arrow key
            i = i + 1
            if i>(len(frames)-2):
                i = len(frames)-2
        prev_frame = frames[i]
    # Release the video capture object
    cv2.destroyAllWindows()
    return D

# Alter Frames

The next fucntions are used to alter the frames in a video.

## Change Color Channel's Ranges

This function changes the range of each individual color channel.

In [21]:
def change_range_colors(image, min_vals=(0, 0, 0), max_vals=(255, 255, 255)):
    # Split the image into its BGR channels
    b, g, r = cv2.split(image)
    # Clip each channel to its respective range
    b = np.clip(b, min_vals[0], max_vals[0])
    g = np.clip(g, min_vals[1], max_vals[1])
    r = np.clip(r, min_vals[2], max_vals[2])
    # Merge the channels back together
    new_image = cv2.merge((b, g, r))
    return new_image

## Add Occlusions

This function adds occlusions in the shape of rectangles and/or circles into the image.

In [22]:
def occlusions(image, num_occlusions=1,loc=[],sizes=[],shapes=['rectangle','circle'], colors=(-1,-1,-1)):    
    output_image = np.copy(image) #Copy of frame
    height, width = image.shape[:2] #Gets size
    num_occlusions = num_occlusions if len(loc)==0 else len(loc) #number of occlusions
    # Draw occlusions on the image
    for i in range(num_occlusions):
        shape_type = random.choice(shapes)
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) if colors==(-1,-1,-1) else colors  # Random color
        if shape_type == 'rectangle':
            x = random.randint(0, width - 1) if len(loc)==0 else loc[i][0]
            y = random.randint(0, height - 1) if len(loc)==0 else loc[i][1]
            width_rect = random.randint(5, width//2) if len(sizes)==0 else loc[i][0]
            height_rect = random.randint(5, height//2) if len(sizes)==0 else loc[i][1]
            cv2.rectangle(output_image, (x, y), (x + width_rect, y + height_rect), color, -1)  # Filled rectangle
        elif shape_type == 'circle':
            center = (random.randint(0, width - 1), random.randint(0, height - 1)) if len(loc)==0 else (loc[i][0],loc[i][1])
            radius = 50#random.randint(5, 100)
            cv2.circle(output_image, center, radius, color, -1)  # Filled circle
    return output_image

## Draw Random Lines

This functions draws straight lines in an image.

In [23]:
def rnd_lines(image, num_lines):
    # Get the dimensions of the image
    height, width = image.shape[:2]
    # Copy the image to avoid modifying the original
    output_image = image.copy()
    for _ in range(num_lines):
        # Generate random start point
        start_point = (random.randint(0, width-1), random.randint(0, height-1))
        # Generate a random angle and length for the line
        angle = random.uniform(0, 2 * np.pi)
        length = random.randint(1, min(width, height) // 2)  # Limit length to half of the smallest dimension
        # Calculate the end point using the angle and length
        end_point = (int(start_point[0] + length * np.cos(angle)), 
                     int(start_point[1] + length * np.sin(angle)))
        # Ensure the end point is within the image boundaries
        end_point = (min(max(end_point[0], 0), width-1), min(max(end_point[1], 0), height-1))
        # Generate a random color (BGR format)
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        # Generate a random thickness for the line
        thickness = random.randint(1, 10)
        # Draw the line on the image
        cv2.line(output_image, start_point, end_point, color, thickness)
    return output_image

## Random Region's Color Change

This function changes the color channel's range in a certain number of rectangle or ellipse regions.

In [24]:
def rnd_regions(image, num_regions):
    # Get the dimensions of the image
    height, width = image.shape[:2]
    # Copy the image to avoid modifying the original
    output_image = image.copy()
    for _ in range(num_regions):
        # Generate random region shape and size
        region_shape = random.choice(['rectangle', 'ellipse'])
        if region_shape == 'rectangle':
            region_width = random.randint(10, width // 3)
            region_height = random.randint(10, height // 3)
            top_left_x = random.randint(0, width - region_width)
            top_left_y = random.randint(0, height - region_height)
            # Define the region
            region = output_image[top_left_y:top_left_y + region_height, top_left_x:top_left_x + region_width]
        elif region_shape == 'ellipse':
            center_x = random.randint(width // 3, width - width // 3)
            center_y = random.randint(height // 3, height - height // 3)
            axis_length = (random.randint(10, width // 3), random.randint(10, height // 3))
            angle = random.randint(0, 360)
            start_angle = 0
            end_angle = 360
            # Create a mask for the ellipse
            mask = np.zeros((height, width), dtype=np.uint8)
            cv2.ellipse(mask, (center_x, center_y), axis_length, angle, start_angle, end_angle, 255, -1)
            # Extract the region using the mask
            region = cv2.bitwise_and(output_image, output_image, mask=mask)
        # Change color channels within the region
        for channel in range(3):  # Assuming BGR format
            # Generate random ranges for the color channel
            low = random.randint(0, 255)
            high = random.randint(low, 255)
            if region_shape == 'rectangle':
                region[..., channel] = np.clip(region[..., channel], low, high)
            elif region_shape == 'ellipse':
                # Apply changes to the region using the mask
                channel_region = output_image[..., channel]
                channel_region[mask == 255] = np.clip(channel_region[mask == 255], low, high)
                output_image[..., channel] = channel_region
        if region_shape == 'rectangle':
            # Place the modified region back into the image for rectangles
            output_image[top_left_y:top_left_y + region_height, top_left_x:top_left_x + region_width] = region
    return output_image

# Simple Image Cartoonization

The next functions are used to cartoonize an image pixel wise by using different algorithms and tecniques.

In [25]:
def cartoonize_image(img, k=10, it = 10, t1 = 150, t2 = 255, ks = 1 , kc=1):
    # Apply bilateral filter to smooth the image
    img_color = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)
    # Convert to grayscale
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply Gaussian Blur
    img_blur = cv2.GaussianBlur(img_gray, (kc, kc), 0)
    # Detect edges using Canny edge detection
    edges = cv2.Canny(img_blur, threshold1=t1, threshold2=t2)
    # Dilate the edges to make them more prominent
    kernel = np.ones((ks, ks), np.uint8)
    edges = cv2.dilate(edges, kernel, iterations=1)
    # Invert the edges
    edges = cv2.bitwise_not(edges)
    # Convert edges back to color, so we can combine with color image
    edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    # Perform K-means clustering
    img_data = np.float32(img_color).reshape((-1, 3))
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, it, 0.2)
    _, labels, centers = cv2.kmeans(img_data, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    centers = np.uint8(centers)
    img_clustered = centers[labels.flatten()]
    img_clustered = img_clustered.reshape(img_color.shape)
    # Combine edge and clustered image
    cartoon = cv2.bitwise_and(img_clustered, edges_colored)
    return cartoon

# Image Variability

The following functions calculate different variability indexes of an image.

In [26]:
def relative_luminance(image):
    b, g, r = cv2.split(image)
    luminance = 0.2126 * r + 0.7152 * g + 0.0722 * b
    #return np.mean(luminance)
    return luminance.sum()/(image.shape[0]*image.shape[1]*255)

def color_variance(image):
    variance_b = np.var(image[:, :, 0])
    variance_g = np.var(image[:, :, 1])
    variance_r = np.var(image[:, :, 2])
    return ((variance_b+variance_g+variance_r)/3)/(image.shape[0]*image.shape[1])

def calculate_variability(image,kernel_size=3):
    kernel = np.ones((kernel_size,kernel_size))/(kernel_size**2-1)
    kernel[kernel_size//2,kernel_size//2] = -1
    V = cv2.filter2D(image,-1,kernel)
    return abs(V).sum()/image.size

# Color Limitation

In [78]:
def color_limit(img,N_colors=20):
    # Convert BGR (OpenCV format) to RGB (PIL format)
    image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Convert NumPy array (RGB) to PIL Image
    pil_image = Image.fromarray(image_rgb)
    # Apply the quantize method in PIL
    quantized_image = pil_image.quantize(N_colors)
    # Convert the quantized PIL image back to NumPy array (RGB format)
    image_rgb_back = np.array(quantized_image.convert('RGB'))  # Convert quantized image back to RGB
    #Convert RGB back to BGR for displaying with OpenCV
    image_bgr_back = cv2.cvtColor(image_rgb_back, cv2.COLOR_RGB2BGR)
    return image_bgr_back

# Pallete Applier

In [91]:
palette = [
    (0, 0, 0),   # Black
    (255, 255, 255),  # White
    (255, 0, 0),   # Red
    (0, 255, 0),   # Green
    (0, 0, 255),   # Blue
    # Add more colors as needed
]

def apply_palette(img, palette):
    img = img.convert("RGB")
    palette_img = Image.new("P", (1, 1))
    palette_img.putpalette(sum(palette, ()))
    return cv2.cvtColor(np.array(img.quantize(palette=palette_img).convert("RGB")),cv2.COLOR_BGR2RGB)

# Add One Inconsistency to Frame

In [ ]:
def AddOneInc(F):
    Bin = np.zeros(F[0].shape[:2],dtype=np.uint8)
    I = F.copy()
    x,y = random.randint(0,2*I.shape[0]//3),random.randint(0,2*I.shape[1]//3)
    l = random.randint(20,I.shape[1]//5)
    Op = random.randint(0,3)
    if Op==0:
        I[x:x+l,y:y+l] = change_range_colors(I[x:x+l,y:y+l],(random.randint(30,140),random.randint(30,140),random.randint(30,140)),(random.randint(150,255),random.randint(150,255),random.randint(150,255)))
        Bin[x:x+l,y:y+l]=255
    elif Op==1:
        R,G,B = random.randint(0,255),random.randint(0,255),random.randint(0,255)
        I[x:x+l,y:y+l] = change_range_colors(I[x:x+l,y:y+l],(R,G,B),(R,G,B))
        Bin[x:x+l,y:y+l]=255
    elif Op==2:
        Thick = random.randint(1,10)
        l2 = random.randint(10,I.shape[1]//8)
        I = cv2.line(I,(x,y),(x+l,y+l2),(random.randint(0,255),random.randint(0,255),random.randint(0,255)),Thick)
        Bin = cv2.line(Bin,(x,y),(x+l,y+l),255,Thick)
    else:
        Thick = random.randint(1,10)
        l2 = random.randint(10,I.shape[1]//8)
        I = cv2.line(I,(x,y),(x+l,y+l2),(0,0,0),Thick)
        Bin = cv2.line(Bin,(x,y),(x+l,y+l),255,Thick)
    return I,Bin

# Add Inconsistencies to Video Each N Frames

In [37]:
def AddInc(F,N=3):
    Inc = []
    Bin = [np.zeros(F[0].shape[:2],dtype=np.uint8) for _ in F]
    for i in range(len(F)):
        if i%N==0 and i!=0:
            I = F[i].copy()
            x,y = random.randint(0,2*I.shape[0]//3),random.randint(0,2*I.shape[1]//3)
            print(I.shape)
            l = random.randint(20,I.shape[1]//5)
            Op = random.randint(0,3)
            if Op==0:
                I[x:x+l,y:y+l] = change_range_colors(I[x:x+l,y:y+l],(random.randint(30,140),random.randint(30,140),random.randint(30,140)),(random.randint(150,255),random.randint(150,255),random.randint(150,255)))
                Bin[i][x:x+l,y:y+l]=255
            elif Op==1:
                R,G,B = random.randint(0,255),random.randint(0,255),random.randint(0,255)
                I[x:x+l,y:y+l] = change_range_colors(I[x:x+l,y:y+l],(R,G,B),(R,G,B))
                Bin[i][x:x+l,y:y+l]=255
            elif Op==2:
                Thick = random.randint(1,10)
                l2 = random.randint(10,I.shape[1]//8)
                I = cv2.line(I,(x,y),(x+l,y+l2),(random.randint(0,255),random.randint(0,255),random.randint(0,255)),Thick)
                Bin[i] = cv2.line(Bin[i],(x,y),(x+l,y+l),255,Thick)
            else:
                Thick = random.randint(1,10)
                l2 = random.randint(10,I.shape[1]//8)
                I = cv2.line(I,(x,y),(x+l,y+l2),(0,0,0),Thick)
                Bin[i] = cv2.line(Bin[i],(x,y),(x+l,y+l),255,Thick)
            Inc.append(I)
        else:
            Inc.append(F[i])
    return Inc,Bin

# Cartoonize Video using K-Means

In [27]:
def cartoonize_vid(F,t1 = 150, t2 = 255, ks = 1 , kc=1):
    C = []
    centers = np.uint8(np.asarray([[r,g,b] for r in range(0,255,40) for g in range(0,255,40) for b in range(0,255,40)]))
    It = 0
    for f in F:
        It = It+1
        print(It,'/',len(F),end='\r')
        img_color = cv2.bilateralFilter(f, d=9, sigmaColor=75, sigmaSpace=75)
        # Convert to grayscale
        img_gray = cv2.cvtColor(f, cv2.COLOR_RGB2GRAY)
        # Apply Gaussian Blur
        img_blur = cv2.GaussianBlur(img_gray, (kc, kc), 0)
        # Detect edges using Canny edge detection
        edges = cv2.Canny(img_blur, threshold1=t1, threshold2=t2)
        # Dilate the edges to make them more prominent
        kernel = np.ones((ks, ks), np.uint8)
        edges = cv2.dilate(edges, kernel, iterations=1)
        # Invert the edges
        edges = cv2.bitwise_not(edges)
        # Convert edges back to color, so we can combine with color image
        edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
        # Perform K-means clustering
        img_data = np.float32(img_color).reshape((-1, 3))

        distances = np.linalg.norm(img_data[:, np.newaxis] - centers, axis=2)
        closest_clusters = np.argmin(distances, axis=1)

        # Map the pixels in the second image to the closest cluster centers
        segmented_pixels2 = centers[closest_clusters]
        segmented_image2 = segmented_pixels2.reshape(f.shape)

        # Combine edge and clustered image
        cartoon = cv2.bitwise_and(segmented_image2, edges_colored)
        C.append(cartoon)
    return C

# Canny Edges

In [28]:
def canny_edge(image,tl=150,th=255,inverted=True):
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply Canny edge detector
    edges = cv2.Canny(gray_image, threshold1=tl, threshold2=th)
    # Invert the binary image (0 becomes 255, and 255 becomes 0)
    if inverted:
        edges = cv2.bitwise_not(edges)
    return edges.astype(np.uint8)

# Sobel Edges

In [29]:
def sobel_edge(image,tl=150,th=255):
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Compute the gradients using the Sobel operator
    sobelx = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)  # Gradient in x-direction
    sobely = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)  # Gradient in y-direction
    # Calculate the magnitude of the gradient
    magnitude = np.sqrt(sobelx**2 + sobely**2)
    # Normalize to range 0 to 255 and convert to uint8
    magnitude = np.uint8(255 * magnitude / np.max(magnitude))
    # Convert to binary image (0 or 255) and invert it
    _, binary_edges = cv2.threshold(magnitude, tl, th, cv2.THRESH_BINARY_INV)
    return binary_edges

# Prewitt Edges

In [30]:
def prewitt_edge(image,tl=150,th=255):
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Define Prewitt kernels
    kernelx = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]], dtype=int)
    kernely = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]], dtype=int)
    # Apply the Prewitt operator
    prewittx = cv2.filter2D(gray_image, -1, kernelx)
    prewitty = cv2.filter2D(gray_image, -1, kernely)
    # Calculate the magnitude of the gradient
    magnitude = np.sqrt(prewittx**2 + prewitty**2)
    # Normalize to range 0 to 255 and convert to uint8
    magnitude = np.uint8(255 * magnitude / np.max(magnitude))
    # Convert to binary image (0 or 255) and invert it
    _, binary_edges = cv2.threshold(magnitude, tl, th, cv2.THRESH_BINARY_INV)
    return binary_edges

# Scharr Edges

In [31]:
def scharr_edge(image,tl=150,th=255):
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Compute the gradients using the Scharr operator
    scharrx = cv2.Scharr(gray_image, cv2.CV_64F, 1, 0)  # Gradient in x-direction
    scharry = cv2.Scharr(gray_image, cv2.CV_64F, 0, 1)  # Gradient in y-direction
    # Calculate the magnitude of the gradient
    magnitude = np.sqrt(scharrx**2 + scharry**2)
    # Normalize to range 0 to 255 and convert to uint8
    magnitude = np.uint8(255 * magnitude / np.max(magnitude))
    # Convert to binary image (0 or 255) and invert it
    _, binary_edges = cv2.threshold(magnitude, tl, th, cv2.THRESH_BINARY_INV)
    return binary_edges

# Laplacian Edges

In [32]:
def laplacian_edge(image,tl=150,th=255):
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply Laplacian operator to find edges
    laplacian_edges = cv2.Laplacian(gray_image, cv2.CV_64F)
    # Convert to absolute values and normalize to range 0 to 255
    laplacian_edges = cv2.convertScaleAbs(laplacian_edges)
    # Convert to binary image (0 or 255) and invert it
    _, binary_edges = cv2.threshold(laplacian_edges, tl, th, cv2.THRESH_BINARY_INV)
    return binary_edges

# Read First N Frames from Videos in Directory

In [33]:
def read_first_frames(directory,N):
    video_frames = []  # Dictionary to store video names and their first 10 frames
    for filename in os.listdir(directory):
        if filename.endswith(".mp4"):  # Check for video file extensions
            video_path = os.path.join(directory, filename)
            cap = cv2.VideoCapture(video_path)
            frames = []
            frame_count = 0
            while frame_count < N and cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break  # If the video ends before 10 frames
                frames.append(frame)
                frame_count += 1
            cap.release()  # Release the video capture object
            video_frames += frames  # Store the frames with the video name
    return video_frames

# Calculate Gradients from 2 Images

In [93]:
def calc_grads(img1, img2):
    # Calculate the spatial gradients using Sobel operator
    gx1 = cv2.Sobel(img1, cv2.CV_64F, 1, 0, ksize=5)
    gy1 = cv2.Sobel(img1, cv2.CV_64F, 0, 1, ksize=5)
    gx2 = cv2.Sobel(img2, cv2.CV_64F, 1, 0, ksize=5)
    gy2 = cv2.Sobel(img2, cv2.CV_64F, 0, 1, ksize=5)
    # Calculate the temporal gradient
    tg = cv2.absdiff(img2, img1)
    # Normalize gradients for display purposes
    gx1d = cv2.convertScaleAbs(gx1)
    gy1d = cv2.convertScaleAbs(gy1)
    gx2d = cv2.convertScaleAbs(gx2)
    gy2d = cv2.convertScaleAbs(gy2)
    tgd = cv2.convertScaleAbs(tg)
    return gx1,gy1,gx2,gy2,tg,gx1d,gy1d,gx2d,gy2d,tgd

# Calculate Movement with Gradients

In [92]:
def calc_grad_movement(img1,img2):
    # Calculate gradients
    gx1,gy1,gx2,gy2,tg,gx1d,gy1d,gx2d,gy2d,tgd = calculate_and_display_gradients(img1, img2)
    # Calculate the magnitude and angle of optical flow
    magnitude = cv2.magnitude(gx2,gy2)
    angle = cv2.phase(gx2,gy2,angleInDegrees=True)
    # Normalize magnitude for display purposes
    magnitude_display = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    magnitude_display = cv2.convertScaleAbs(magnitude_display)
    # Optional: Visualize flow direction using HSV color space
    hsv = np.zeros((img1.shape[0], img1.shape[1], 3), dtype=np.uint8)
    hsv[..., 1] = 255
    hsv[..., 0] = angle / 2
    hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    optical_flow = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return magnitude_display,optical_flow

# Calculate Frames Differences with 3 Channels

In [95]:
def calc_differences(img1,img2):
    # Convert images to RGB (OpenCV loads images in BGR format by default)
    img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    # Convert images to grayscale
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    # Calculate the absolute difference between the images
    diff_rgb = np.abs(img1_rgb - img2_rgb)
    diff_gray = np.abs(img1_gray.astype(np.float32) - img2_gray.astype(np.float32))
    # Normalize the differences to [0, 255]
    norm_diff_rgb = cv2.normalize(diff_rgb, None, 0, 255, cv2.NORM_MINMAX)
    norm_diff_gray = cv2.normalize(diff_gray, None, 0, 255, cv2.NORM_MINMAX)
    # Split the RGB difference into color channels
    diff_r, diff_g, diff_b = cv2.split(norm_diff_rgb)
    # Convert the single channel differences back to 3-channel images for display
    diff_r_colored = cv2.merge([diff_r, np.zeros_like(diff_r), np.zeros_like(diff_r)])
    diff_g_colored = cv2.merge([np.zeros_like(diff_g), diff_g, np.zeros_like(diff_g)])
    diff_b_colored = cv2.merge([np.zeros_like(diff_b), np.zeros_like(diff_b), diff_b])
    return diff_r_colored,diff_g_colored,diff_b_colored,norm_diff_gray.astype(np.uint8)

# Calculate Temporal Gradient for each Color Channel

In [97]:
def calc_temporal_gradient(img1, img2):
    # Convert images to RGB (OpenCV loads images in BGR format by default)
    img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    # Convert images to grayscale
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    # Calculate the temporal gradient (difference) between the images
    grad_rgb = np.abs(img2_rgb - img1_rgb)
    grad_gray = np.abs(img2_gray.astype(np.float32) - img1_gray.astype(np.float32))
    # Normalize the gradients to [0, 255]
    norm_grad_rgb = cv2.normalize(grad_rgb, None, 0, 255, cv2.NORM_MINMAX)
    norm_grad_gray = cv2.normalize(grad_gray, None, 0, 255, cv2.NORM_MINMAX)
    # Split the RGB gradient into color channels
    grad_r, grad_g, grad_b = cv2.split(norm_grad_rgb)
    # Convert the single channel gradients back to 3-channel images for display
    grad_r_colored = cv2.merge([grad_r, np.zeros_like(grad_r), np.zeros_like(grad_r)])
    grad_g_colored = cv2.merge([np.zeros_like(grad_g), grad_g, np.zeros_like(grad_g)])
    grad_b_colored = cv2.merge([np.zeros_like(grad_b), np.zeros_like(grad_b), grad_b])
    return grad_r_colored,grad_g_colored,grad_b_colored,norm_grad_gray.astype(np.uint8)

# Temporal Consistency Index

In [ ]:
def calc_temporal_consistency(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        raise ValueError("Error opening video file. Check the file path.")
    
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_count < 2:
        raise ValueError("Video must have at least two frames to calculate temporal consistency.")
    
    # Initialize variables to accumulate error values
    total_error_rgb = np.zeros(3)  # R, G, B channels
    total_error_gray = 0
    num_frames = 0

    # Read the first frame
    ret, prev_frame = cap.read()
    if not ret:
        raise ValueError("Failed to read the first frame.")
    
    prev_frame_rgb = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2RGB)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    
    while True:
        # Read the next frame
        ret, curr_frame = cap.read()
        if not ret:
            break
        
        curr_frame_rgb = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2RGB)
        curr_frame_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        
        # Compute the absolute difference between consecutive frames
        diff_rgb = np.abs(curr_frame_rgb.astype(np.float32) - prev_frame_rgb.astype(np.float32))
        diff_gray = np.abs(curr_frame_gray.astype(np.float32) - prev_frame_gray.astype(np.float32))
        
        # Calculate the mean absolute error for the current frame pair
        mean_error_rgb = np.mean(diff_rgb, axis=(0, 1))  # Mean error for R, G, B channels
        mean_error_gray = np.mean(diff_gray)
        
        total_error_rgb += mean_error_rgb
        total_error_gray += mean_error_gray
        num_frames += 1
        
        # Update previous frame
        prev_frame_rgb = curr_frame_rgb
        prev_frame_gray = curr_frame_gray
    
    # Compute the average temporal consistency index
    if num_frames == 0:
        raise ValueError("No frames processed. Check the video file.")
    
    temporal_consistency_index_rgb = total_error_rgb / num_frames
    temporal_consistency_index_gray = total_error_gray / num_frames
    
    # Release video capture object
    cap.release()
    
    return temporal_consistency_index_rgb, temporal_consistency_index_gray

In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.color import rgb2gray

def calculate_temporal_gradient(image1, image2):
    # Convert images to RGB and grayscale
    img1_rgb = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    img1_gray = rgb2gray(img1_rgb)
    img2_gray = rgb2gray(img2_rgb)

    # Calculate the temporal gradient (difference) for each channel and grayscale
    grad_rgb = np.abs(img2_rgb.astype(np.float32) - img1_rgb.astype(np.float32))
    grad_gray = np.abs(img2_gray.astype(np.float32) - img1_gray.astype(np.float32))

    return grad_rgb, grad_gray

def calculate_similarity_indexes(image1, image2):
    # Convert images to RGB and grayscale
    img1_rgb = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    img1_gray = rgb2gray(img1_rgb)
    img2_gray = rgb2gray(img2_rgb)
    
    # Resize images to the same size if they are different
    if img1_rgb.shape != img2_rgb.shape:
        img2_rgb = cv2.resize(img2_rgb, (img1_rgb.shape[1], img1_rgb.shape[0]))
        img2_gray = cv2.resize(img2_gray, (img1_gray.shape[1], img1_gray.shape[0]))

    # Compute Mean Squared Error (MSE) for RGB
    mse_rgb = np.mean((img1_rgb - img2_rgb) ** 2)
    
    # Compute Structural Similarity Index (SSIM) for grayscale
    ssim_gray, _ = ssim(img1_gray, img2_gray, full=True)
    
    # Compute Color Histograms for RGB
    hist1_r = cv2.calcHist([img1_rgb[:, :, 0]], [0], None, [256], [0, 256])
    hist2_r = cv2.calcHist([img2_rgb[:, :, 0]], [0], None, [256], [0, 256])
    hist1_g = cv2.calcHist([img1_rgb[:, :, 1]], [0], None, [256], [0, 256])
    hist2_g = cv2.calcHist([img2_rgb[:, :, 1]], [0], None, [256], [0, 256])
    hist1_b = cv2.calcHist([img1_rgb[:, :, 2]], [0], None, [256], [0, 256])
    hist2_b = cv2.calcHist([img2_rgb[:, :, 2]], [0], None, [256], [0, 256])
    
    hist_corr_r = cv2.compareHist(hist1_r, hist2_r, cv2.HISTCMP_CORREL)
    hist_corr_g = cv2.compareHist(hist1_g, hist2_g, cv2.HISTCMP_CORREL)
    hist_corr_b = cv2.compareHist(hist1_b, hist2_b, cv2.HISTCMP_CORREL)
    
    # Compute Edge-based similarity (using Canny edge detector)
    edges1 = cv2.Canny(cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY), 100, 200)
    edges2 = cv2.Canny(cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY), 100, 200)
    
    edge_similarity = np.sum(edges1 == edges2) / np.size(edges1)
    
    return {
        'MSE_RGB': mse_rgb,
        'SSIM_Gray': ssim_gray,
        'Hist_Corr_R': hist_corr_r,
        'Hist_Corr_G': hist_corr_g,
        'Hist_Corr_B': hist_corr_b,
        'Edge_Similarity': edge_similarity
    }

def identify_abnormality(image1_path, image2_path):
    # Read the images
    img1 = cv2.imread(image1_path)
    img2 = cv2.imread(image2_path)
    
    if img1 is None or img2 is None:
        raise ValueError("One or both images could not be loaded. Check the file paths.")
    
    # Calculate temporal gradients
    grad_rgb, grad_gray = calculate_temporal_gradient(img1, img2)
    
    # Calculate similarity indexes
    similarity_indexes = calculate_similarity_indexes(img1, img2)
    
    # Combine gradients and similarity indexes to identify abnormalities
    # Normalize gradients for visibility
    norm_grad_rgb = cv2.normalize(grad_rgb, None, 0, 255, cv2.NORM_MINMAX)
    norm_grad_gray = cv2.normalize(grad_gray, None, 0, 255, cv2.NORM_MINMAX)
    
    # Create a mask where significant changes or low similarity occur
    threshold = 50  # Set an appropriate threshold value
    abnormal_rgb_mask = np.max(norm_grad_rgb, axis=2) > threshold
    abnormal_gray_mask = norm_grad_gray > threshold

    # Highlight the regions of interest
    highlighted_rgb = img1.copy()
    highlighted_rgb[abnormal_rgb_mask] = [0, 0, 255]  # Mark changes in red
    highlighted_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    highlighted_gray[abnormal_gray_mask] = 255  # Mark changes in white
    
    # Display the results
    cv2.imshow('Original Image 1', img1)
    cv2.imshow('Original Image 2', img2)
    cv2.imshow('Temporal Gradient (RGB)', norm_grad_rgb.astype(np.uint8))
    cv2.imshow('Temporal Gradient (Gray)', norm_grad_gray.astype(np.uint8))
    cv2.imshow('Abnormal Regions (RGB)', highlighted_rgb)
    cv2.imshow('Abnormal Regions (Gray)', highlighted_gray)
    
    # Print similarity indexes
    print(f'Similarity Indexes: {similarity_indexes}')
    
    # Wait until a key is pressed and close the windows
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
#image1_path = 'saved_frames/frame_000.png'
#image2_path = 'saved_frames/frame_001.png'
#identify_abnormality(image1_path, image2_path)


In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.color import rgb2gray

def calculate_temporal_gradient(image1, image2):
    # Convert images to RGB and grayscale
    img1_rgb = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    img1_gray = rgb2gray(img1_rgb)
    img2_gray = rgb2gray(img2_rgb)

    # Calculate the temporal gradient (difference) for each channel and grayscale
    grad_rgb = np.abs(img2_rgb.astype(np.float32) - img1_rgb.astype(np.float32))
    grad_gray = np.abs(img2_gray.astype(np.float32) - img1_gray.astype(np.float32))

    return grad_rgb, grad_gray

def calculate_similarity_indexes(image1, image2):
    # Convert images to RGB and grayscale
    img1_rgb = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    img1_gray = rgb2gray(img1_rgb)
    img2_gray = rgb2gray(img2_rgb)
    
    # Resize images to the same size if they are different
    if img1_rgb.shape != img2_rgb.shape:
        img2_rgb = cv2.resize(img2_rgb, (img1_rgb.shape[1], img1_rgb.shape[0]))
        img2_gray = cv2.resize(img2_gray, (img1_gray.shape[1], img1_gray.shape[0]))

    # Compute Mean Squared Error (MSE) for RGB
    mse_rgb = np.mean((img1_rgb - img2_rgb) ** 2)
    
    # Compute Structural Similarity Index (SSIM) for grayscale
    ssim_gray, _ = ssim(img1_gray, img2_gray, full=True)
    
    # Compute Color Histograms for RGB
    hist1_r = cv2.calcHist([img1_rgb[:, :, 0]], [0], None, [256], [0, 256])
    hist2_r = cv2.calcHist([img2_rgb[:, :, 0]], [0], None, [256], [0, 256])
    hist1_g = cv2.calcHist([img1_rgb[:, :, 1]], [0], None, [256], [0, 256])
    hist2_g = cv2.calcHist([img2_rgb[:, :, 1]], [0], None, [256], [0, 256])
    hist1_b = cv2.calcHist([img1_rgb[:, :, 2]], [0], None, [256], [0, 256])
    hist2_b = cv2.calcHist([img2_rgb[:, :, 2]], [0], None, [256], [0, 256])
    
    hist_corr_r = cv2.compareHist(hist1_r, hist2_r, cv2.HISTCMP_CORREL)
    hist_corr_g = cv2.compareHist(hist1_g, hist2_g, cv2.HISTCMP_CORREL)
    hist_corr_b = cv2.compareHist(hist1_b, hist2_b, cv2.HISTCMP_CORREL)
    
    # Compute Edge-based similarity (using Canny edge detector)
    edges1 = cv2.Canny(cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY), 100, 200)
    edges2 = cv2.Canny(cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY), 100, 200)
    
    edge_similarity = np.sum(edges1 == edges2) / np.size(edges1)
    
    return {
        'MSE_RGB': mse_rgb,
        'SSIM_Gray': ssim_gray,
        'Hist_Corr_R': hist_corr_r,
        'Hist_Corr_G': hist_corr_g,
        'Hist_Corr_B': hist_corr_b,
        'Edge_Similarity': edge_similarity
    }

def process_video(video_path, output_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Error opening video file. Check the file path.")
    
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Define codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Using MP4 codec
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width * 2, frame_height * 2))  # Updated size for combined frames
    
    ret, prev_frame = cap.read()
    if not ret:
        raise ValueError("Failed to read the first frame.")
    
    prev_frame_rgb = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2RGB)
    prev_frame_gray = rgb2gray(prev_frame_rgb)
    
    while True:
        ret, curr_frame = cap.read()
        if not ret:
            break
        
        curr_frame_rgb = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2RGB)
        curr_frame_gray = rgb2gray(curr_frame_rgb)
        
        # Calculate temporal gradients and similarity indexes
        grad_rgb, grad_gray = calculate_temporal_gradient(prev_frame, curr_frame)
        similarity_indexes = calculate_similarity_indexes(prev_frame, curr_frame)
        
        # Normalize gradients for visibility
        norm_grad_rgb = cv2.normalize(grad_rgb, None, 0, 255, cv2.NORM_MINMAX)
        norm_grad_gray = cv2.normalize(grad_gray, None, 0, 255, cv2.NORM_MINMAX)
        
        # Create masks for abnormalities
        threshold = 50  # Set an appropriate threshold value
        abnormal_rgb_mask = np.max(norm_grad_rgb, axis=2) > threshold
        abnormal_gray_mask = norm_grad_gray > threshold

        # Highlight the regions of interest
        highlighted_rgb = curr_frame.copy()
        highlighted_rgb[abnormal_rgb_mask] = [0, 0, 255]  # Mark changes in red
        highlighted_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        highlighted_gray[abnormal_gray_mask] = 255  # Mark changes in white
        
        # Combine the results into a single frame
        combined_frame = np.hstack((curr_frame, cv2.cvtColor(highlighted_rgb, cv2.COLOR_BGR2RGB)))
        combined_frame = np.vstack((combined_frame, np.hstack((cv2.cvtColor(highlighted_gray, cv2.COLOR_GRAY2BGR), norm_grad_rgb.astype(np.uint8)))))
        
        # Write the combined frame to the output video
        out.write(combined_frame)
        
        # Update previous frame
        prev_frame_rgb = curr_frame_rgb
        prev_frame_gray = curr_frame_gray
    
    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Example usage
#video_path = 'Cartoonized/U_toon.mp4'
#output_path = 'output_abnormalities.mp4'
#process_video(video_path, output_path)


In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.color import rgb2gray

def calculate_temporal_gradient(image1, image2):
    # Convert images to RGB and grayscale
    img1_rgb = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    img1_gray = rgb2gray(img1_rgb)
    img2_gray = rgb2gray(img2_rgb)

    # Calculate the temporal gradient (difference) for each channel and grayscale
    grad_rgb = np.abs(img2_rgb.astype(np.float32) - img1_rgb.astype(np.float32))
    grad_gray = np.abs(img2_gray.astype(np.float32) - img1_gray.astype(np.float32))

    return grad_rgb, grad_gray

def calculate_similarity_indexes(image1, image2):
    # Convert images to RGB and grayscale
    img1_rgb = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    img1_gray = rgb2gray(img1_rgb)
    img2_gray = rgb2gray(img2_rgb)
    
    # Resize images to the same size if they are different
    if img1_rgb.shape != img2_rgb.shape:
        img2_rgb = cv2.resize(img2_rgb, (img1_rgb.shape[1], img1_rgb.shape[0]))
        img2_gray = cv2.resize(img2_gray, (img1_gray.shape[1], img1_gray.shape[0]))

    # Compute Mean Squared Error (MSE) for RGB
    mse_rgb = np.mean((img1_rgb - img2_rgb) ** 2)
    
    # Compute Structural Similarity Index (SSIM) for grayscale
    ssim_gray, _ = ssim(img1_gray, img2_gray, full=True)
    
    # Compute Color Histograms for RGB
    hist1_r = cv2.calcHist([img1_rgb[:, :, 0]], [0], None, [256], [0, 256])
    hist2_r = cv2.calcHist([img2_rgb[:, :, 0]], [0], None, [256], [0, 256])
    hist1_g = cv2.calcHist([img1_rgb[:, :, 1]], [0], None, [256], [0, 256])
    hist2_g = cv2.calcHist([img2_rgb[:, :, 1]], [0], None, [256], [0, 256])
    hist1_b = cv2.calcHist([img1_rgb[:, :, 2]], [0], None, [256], [0, 256])
    hist2_b = cv2.calcHist([img2_rgb[:, :, 2]], [0], None, [256], [0, 256])
    
    hist_corr_r = cv2.compareHist(hist1_r, hist2_r, cv2.HISTCMP_CORREL)
    hist_corr_g = cv2.compareHist(hist1_g, hist2_g, cv2.HISTCMP_CORREL)
    hist_corr_b = cv2.compareHist(hist1_b, hist2_b, cv2.HISTCMP_CORREL)
    
    # Compute Edge-based similarity (using Canny edge detector)
    edges1 = cv2.Canny(cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY), 100, 200)
    edges2 = cv2.Canny(cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY), 100, 200)
    
    edge_similarity = np.sum(edges1 == edges2) / np.size(edges1)
    
    return {
        'MSE_RGB': mse_rgb,
        'SSIM_Gray': ssim_gray,
        'Hist_Corr_R': hist_corr_r,
        'Hist_Corr_G': hist_corr_g,
        'Hist_Corr_B': hist_corr_b,
        'Edge_Similarity': edge_similarity
    }

def add_label(image, text, position=(10, 30), font_scale=1, color=(255, 255, 255), thickness=2):
    """Add a label to an image."""
    return cv2.putText(image, text, position, cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness)

def process_video(video_path, output_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Error opening video file. Check the file path.")
    
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Define codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Using MP4 codec
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width * 3, frame_height * 2))  # Updated size for combined frames
    
    ret, prev_frame = cap.read()
    if not ret:
        raise ValueError("Failed to read the first frame.")
    
    prev_frame_rgb = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2RGB)
    prev_frame_gray = rgb2gray(prev_frame_rgb)
    
    while True:
        ret, curr_frame = cap.read()
        if not ret:
            break
        
        curr_frame_rgb = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2RGB)
        curr_frame_gray = rgb2gray(curr_frame_rgb)
        
        # Calculate temporal gradients and similarity indexes
        grad_rgb, grad_gray = calculate_temporal_gradient(prev_frame, curr_frame)
        similarity_indexes = calculate_similarity_indexes(prev_frame, curr_frame)
        
        # Normalize gradients for visibility
        norm_grad_rgb = cv2.normalize(grad_rgb, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        norm_grad_gray = cv2.normalize(grad_gray, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        
        # Convert single-channel grayscale to 3-channel RGB for displaying
        norm_grad_gray_rgb = cv2.cvtColor(norm_grad_gray, cv2.COLOR_GRAY2RGB)
        
        # Create masks for abnormalities
        threshold = 50  # Set an appropriate threshold value
        abnormal_rgb_mask = np.max(norm_grad_rgb, axis=2) > threshold
        abnormal_gray_mask = norm_grad_gray > threshold

        # Highlight the regions of interest
        highlighted_rgb = curr_frame.copy()
        highlighted_rgb[abnormal_rgb_mask] = [0, 0, 255]  # Mark changes in red
        highlighted_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        highlighted_gray[abnormal_gray_mask] = 255  # Mark changes in white
        
        # Create the combined frame
        combined_frame = np.zeros((frame_height * 2, frame_width * 3, 3), dtype=np.uint8)
        combined_frame[0:frame_height, 0:frame_width] = prev_frame
        combined_frame[0:frame_height, frame_width:frame_width*2] = curr_frame
        combined_frame[0:frame_height, frame_width*2:frame_width*3] = highlighted_rgb
        combined_frame[frame_height:frame_height*2, 0:frame_width] = cv2.cvtColor(highlighted_gray, cv2.COLOR_GRAY2BGR)
        combined_frame[frame_height:frame_height*2, frame_width:frame_width*2] = norm_grad_rgb
        combined_frame[frame_height:frame_height*2, frame_width*2:frame_width*3] = norm_grad_gray_rgb
        
        # Add labels to the combined frame
        combined_frame = add_label(combined_frame, "Previous Frame", position=(10, 30))
        combined_frame = add_label(combined_frame, "Current Frame", position=(frame_width + 10, 30))
        combined_frame = add_label(combined_frame, "Highlighted Changes", position=(frame_width * 2 + 10, 30))
        combined_frame = add_label(combined_frame, "Gray Gradient", position=(10, frame_height + 30))
        combined_frame = add_label(combined_frame, "RGB Gradient", position=(frame_width + 10, frame_height + 30))
        combined_frame = add_label(combined_frame, "Gray Gradient (Norm)", position=(frame_width * 2 + 10, frame_height + 30))
        
        # Write the combined frame to the output video
        out.write(combined_frame)
        
        # Update previous frame
        prev_frame = curr_frame
        prev_frame_rgb = curr_frame_rgb
        prev_frame_gray = curr_frame_gray
    
    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Example usage
#video_path = 'Cartoonized/U_toon.mp4'
#output_path = 'output_abnormalities1.mp4'
#process_video(video_path, output_path)


# Random Region Warping to image (Deformation)

In [102]:
def img_deformation(image, max_movement=10, smoothness=50):
    # Get the height and width of the image
    h, w, _ = image.shape
    # Generate random noise-based flow fields for X and Y directions
    flow_X = np.random.uniform(-1, 1, (h, w)).astype(np.float32)
    flow_Y = np.random.uniform(-1, 1, (h, w)).astype(np.float32)
    # Smooth the noise to ensure consistency across regions
    flow_X = gaussian_filter(flow_X, sigma=smoothness)
    flow_Y = gaussian_filter(flow_Y, sigma=smoothness)
    # Normalize the flow fields to the range of [-max_movement, max_movement]
    flow_X = cv2.normalize(flow_X, None, -max_movement, max_movement, cv2.NORM_MINMAX)
    flow_Y = cv2.normalize(flow_Y, None, -max_movement, max_movement, cv2.NORM_MINMAX)
    # Generate a meshgrid of coordinates (X, Y)
    X, Y = np.meshgrid(np.arange(w), np.arange(h))
    # Create a map that distorts the image based on the smoothed optical flow
    map_X = (X + flow_X).astype(np.float32)
    map_Y = (Y + flow_Y).astype(np.float32)
    # Warp the image using remap function
    warped_image = cv2.remap(image, map_X, map_Y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
    return warped_image

# Random Point Warping

In [110]:
def random_point_warp(image, max_movement=1):
    # Get the height and width of the image
    h, w, _ = image.shape
    # Generate a meshgrid of coordinates (X, Y)
    X, Y = np.meshgrid(np.arange(w), np.arange(h))
    # Generate random optical flow for X and Y directions
    flow_X = np.random.uniform(-max_movement, max_movement, (h, w)).astype(np.float32)
    flow_Y = np.random.uniform(-max_movement, max_movement, (h, w)).astype(np.float32)
    # Create a map that distorts the image based on the optical flow
    map_X = (X + flow_X).astype(np.float32)
    map_Y = (Y + flow_Y).astype(np.float32)
    # Warp the image using remap function
    warped_image = cv2.remap(image, map_X, map_Y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
    return warped_image

# Random Perspective, Rotation and Resize

In [119]:
def random_PRR(image, max_shift=5, scale_range=(0.9, 1.1), max_angle=5):
    # Get the height and width of the original image
    h, w, _ = image.shape
    # ----- Resizing -----
    # Choose a random scale factor within the specified range
    scale_factor = np.random.uniform(scale_range[0], scale_range[1])
    # Compute new dimensions based on the scale factor
    new_w, new_h = int(w * scale_factor), int(h * scale_factor)
    # Resize the image
    resized_image = cv2.resize(image, (new_w, new_h))
    # ---- Rotation -----
    # Choose a random angle for rotation (within max_angle degrees)
    angle = np.random.uniform(-max_angle, max_angle)
    # Get the rotation matrix (centered at the new image's center)
    center = (new_w // 2, new_h // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    # Rotate the resized image
    rotated_image = cv2.warpAffine(resized_image, rotation_matrix, (new_w, new_h))
    # ----- Perspective Warp -----
    # Define the original corner points of the rotated image
    src_points = np.float32([[0, 0], [new_w, 0], [new_w, new_h], [0, new_h]])
    # Generate small random shifts for each corner
    dst_points = src_points + np.random.randint(-max_shift, max_shift, src_points.shape).astype(np.float32)
    # Get the perspective transformation matrix
    M = cv2.getPerspectiveTransform(src_points, dst_points)
    # Apply the warp to the rotated image
    warped_image = cv2.warpPerspective(rotated_image, M, (new_w, new_h))
    return warped_image

# Entropy

In [ ]:
@numba.jit
def Entropy(image):
    # Convert the image to grayscale using manual conversion
    gray_image = 0.299 * image[:, :, 2] + 0.587 * image[:, :, 1] + 0.114 * image[:, :, 0]
    # Calculate the histogram of the grayscale image
    hist, _ = np.histogram(gray_image, bins=256, range=(0, 256), density=True)
    # Avoid using SciPy's entropy function, instead calculate it manually
    hist = hist[hist > 0]  # Remove zero entries to avoid log(0)
    hist_entropy = -np.sum(hist * np.log2(hist))
    return hist_entropy

# TSNR

In [ ]:
@numba.jit
def TSNR(image1, image2, image3):  
    if image1 is None or image2 is None:
        raise ValueError("One or both images are invalid")   
    # Calculate the absolute difference manually
    frame_diff = np.abs(image1.astype(np.float64) - image2.astype(np.float64))
    # Calculate the mean and standard deviation of the difference
    mean_diff = np.mean(frame_diff)
    std_diff = np.std(frame_diff)
    # Compute TSNR
    tsnr = mean_diff / (std_diff + 1e-10)
    return abs(1 - tsnr)

## Absolute Difference

In [ ]:
@numba.jit
def Abs_Dif(image1, image2, image3):
    return np.mean(np.abs(image1 - image2))

# Optical Flow End Point Error

In [ ]:
@numba.jit
def OF_EPE(image1, image2, image3):
    if image1 is None or image2 is None:
        raise ValueError("One or both images are invalid")
    # Manual grayscale conversion (approximating cv2.cvtColor)
    gray1 = 0.299 * image1[:, :, 2] + 0.587 * image1[:, :, 1] + 0.114 * image1[:, :, 0]
    gray2 = 0.299 * image2[:, :, 2] + 0.587 * image2[:, :, 1] + 0.114 * image2[:, :, 0]
    # Placeholder for optical flow (since Farneback can't be done with NumPy)
    # For real use, replace this with actual optical flow computation outside of Numba.
    # Example: Use np.gradient or any other available method that approximates movement.
    flow_x = np.gradient(gray2, axis=1)  # Approximation: gradient as optical flow
    flow_y = np.gradient(gray2, axis=0)
    flow = np.stack((flow_x, flow_y), axis=-1)
    # EPE (End Point Error) calculation
    epe = np.linalg.norm(flow, axis=2).mean()
    return epe

# Oprical Flow Angular Error

In [ ]:
@numba.jit
def OF_AE(image1, image2, image3):
    if image1 is None or image2 is None:
        raise ValueError("One or both images are invalid")
    # Manual grayscale conversion (approximation of cv2.cvtColor)
    gray1 = 0.299 * image1[:, :, 2] + 0.587 * image1[:, :, 1] + 0.114 * image1[:, :, 0]
    gray2 = 0.299 * image2[:, :, 2] + 0.587 * image2[:, :, 1] + 0.114 * image2[:, :, 0] 
    # Placeholder for optical flow (since Farneback can't be done with NumPy)
    # Example: Gradient approximation for optical flow
    flow_x = np.gradient(gray2, axis=1)  # Approximation: gradient as optical flow
    flow_y = np.gradient(gray2, axis=0)
    u, v = flow_x, flow_y   
    # Magnitude and angle calculation (approximation of cv2.cartToPolar)
    magnitude = np.sqrt(u**2 + v**2)
    angle = np.arctan2(v, u)  
    # AE (Angular Error) calculation
    ae = np.mean(np.abs(angle))  # Taking the mean absolute angle error  
    return ae

# Optical Flow Error

In [ ]:
@numba.jit
def OF(img1, img2, img3):
    # Check if images are the same size
    if img1.shape != img2.shape or img2.shape != img3.shape:
        raise ValueError("All images must be of the same size")
    # Convert to grayscale using a simple weighted sum
    edges1 = 0.299 * img1[:, :, 2] + 0.587 * img1[:, :, 1] + 0.114 * img1[:, :, 0]
    edges2 = 0.299 * img2[:, :, 2] + 0.587 * img2[:, :, 1] + 0.114 * img2[:, :, 0]
    edges3 = 0.299 * img3[:, :, 2] + 0.587 * img3[:, :, 1] + 0.114 * img3[:, :, 0]
    # Approximate optical flow by using gradients (placeholder)
    flow1_x = np.gradient(edges2, axis=1) - np.gradient(edges1, axis=1)
    flow1_y = np.gradient(edges2, axis=0) - np.gradient(edges1, axis=0)
    flow2_x = np.gradient(edges3, axis=1) - np.gradient(edges2, axis=1)
    flow2_y = np.gradient(edges3, axis=0) - np.gradient(edges2, axis=0)
    # Compute the magnitude and angle of the optical flow vectors for both flows
    magnitude1 = np.sqrt(flow1_x**2 + flow1_y**2)
    angle1 = np.arctan2(flow1_y, flow1_x)
    magnitude2 = np.sqrt(flow2_x**2 + flow2_y**2)
    angle2 = np.arctan2(flow2_y, flow2_x)
    # Compute the difference between the two optical flows
    magnitude_diff = np.abs(magnitude1 - magnitude2)
    angle_diff = np.abs(angle1 - angle2)
    # Normalize differences
    magnitude_index = np.sum(magnitude_diff) / (magnitude1.size + 1e-6)  # Avoid division by zero
    angle_index = np.sum(angle_diff) / (angle1.size + 1e-6)  # Avoid division by zero
    # Combine magnitude and angle differences
    difference_index = magnitude_index + angle_index
    return difference_index

# Optical Flow Border Inconsistency Index

In [ ]:
@numba.jit
def OF_B(img1, img2, img3):
    # Check if images are the same size
    if img1.shape != img2.shape or img2.shape != img3.shape:
        raise ValueError("All images must be of the same size")
    # Simple edge detection approximation using gradients
    def edge_detect(image):
        dx = np.abs(np.gradient(image, axis=1))
        dy = np.abs(np.gradient(image, axis=0))
        return np.sqrt(dx**2 + dy**2)  
    # Apply the edge detection approximation
    edges1 = edge_detect(0.299 * img1[:, :, 2] + 0.587 * img1[:, :, 1] + 0.114 * img1[:, :, 0])
    edges2 = edge_detect(0.299 * img2[:, :, 2] + 0.587 * img2[:, :, 1] + 0.114 * img2[:, :, 0])
    edges3 = edge_detect(0.299 * img3[:, :, 2] + 0.587 * img3[:, :, 1] + 0.114 * img3[:, :, 0])
    # Approximate optical flow by using gradients (placeholder)
    flow1_x = np.gradient(edges2, axis=1) - np.gradient(edges1, axis=1)
    flow1_y = np.gradient(edges2, axis=0) - np.gradient(edges1, axis=0)
    flow2_x = np.gradient(edges3, axis=1) - np.gradient(edges2, axis=1)
    flow2_y = np.gradient(edges3, axis=0) - np.gradient(edges2, axis=0)
    # Compute the magnitude and angle of the optical flow vectors for both flows
    magnitude1 = np.sqrt(flow1_x**2 + flow1_y**2)
    angle1 = np.arctan2(flow1_y, flow1_x)
    magnitude2 = np.sqrt(flow2_x**2 + flow2_y**2)
    angle2 = np.arctan2(flow2_y, flow2_x)
    # Compute the difference between the two optical flows
    magnitude_diff = np.abs(magnitude1 - magnitude2)
    angle_diff = np.abs(angle1 - angle2)
    # Normalize differences
    magnitude_index = np.sum(magnitude_diff) / (magnitude1.size + 1e-6)  # Avoid division by zero
    angle_index = np.sum(angle_diff) / (angle1.size + 1e-6)  # Avoid division by zero
    # Combine magnitude and angle differences
    difference_index = magnitude_index + angle_index
    return difference_index

# Gray Scale Absolute Difference

In [ ]:
@numba.jit
def Gray_Dif(image1, image2, image3):
    def rgb_to_gray(image):
        return (image[:, :, 2] + image[:, :, 1] + image[:, :, 0])/3
    gray1 = rgb_to_gray(image1)
    gray2 = rgb_to_gray(image2)
    # Calculate the absolute difference between the two grayscale images
    diff = np.abs(gray1 - gray2)
    # Return the mean of the difference
    return np.mean(diff)

# Temporal Structural Similarity Index Measure for Inconsistency

In [120]:
@numba.jit
def TSSIM(image1, image2, image3):
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid")
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    ssim_value = ssim(gray1, gray2,multichannel=True,win_size=3)
    return abs(1-ssim_value)

# Mean Squared Error

In [ ]:
@numba.jit
def MSE(image1, image2, image3):
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid")
    # Convert images to grayscale
    def rgb_to_gray(image):
        return 0.299 * image[:, :, 2] + 0.587 * image[:, :, 1] + 0.114 * image[:, :, 0]
    gray1 = rgb_to_gray(image1)
    gray2 = rgb_to_gray(image2)
    # Compute the Mean Squared Error
    mse = np.mean((gray1 - gray2) ** 2)
    return mse

# Border Error

In [ ]:
@numba.jit
def Border_Err(image1, image2, image3):
    lower_threshold = 50
    upper_threshold = 255
    if len(image1.shape) > 1:
        image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    if len(image2.shape) > 1:
        image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    # Apply Canny edge detection
    canny1 = cv2.Canny(image1, lower_threshold, upper_threshold)
    canny2 = cv2.Canny(image2, lower_threshold, upper_threshold)
    # Calculate the absolute difference between the two Canny images
    abs_diff = cv2.absdiff(canny1, canny2)
    return np.mean(abs_diff)

# Color Range Consistency

In [ ]:
@numba.jit
def CRC(image1, image2, image3):
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid")    
    # Convert images to float64
    image1 = image1.astype(np.float64)
    image2 = image2.astype(np.float64)  
    # Calculate differences for each channel
    def channel_diff(image1, image2, channel):
        min1 = np.min(image1[:,:,channel])
        max1 = np.max(image1[:,:,channel])
        min2 = np.min(image2[:,:,channel])
        max2 = np.max(image2[:,:,channel])
        diff_min = abs(min1 - min2)
        diff_max = abs(max1 - max2)
        return diff_min, diff_max   
    diff_r_min, diff_r_max = channel_diff(image1, image2, 2)  # Red channel
    diff_g_min, diff_g_max = channel_diff(image1, image2, 1)  # Green channel
    diff_b_min, diff_b_max = channel_diff(image1, image2, 0)  # Blue channel   
    # Calculate CRC index
    crci = (diff_r_min + diff_r_max + diff_g_min + diff_g_max + diff_b_min + diff_b_max) / 6    
    return crci

# Entropy Difference

In [ ]:
@numba.numba.jit
def Entropy_Dif(image1,image2, image3):
    return abs(Entropy(image1)-Entropy(image2))

# Frequency Magnitude Difference

In [ ]:
@numba.jit
def Freq_Dif(img1, img2, img3):
    # Check if images are the same size
    if img1.shape != img2.shape:
        raise ValueError("Images must be of the same size")  
    # Compute FFT
    f1 = np.fft.fft2(img1)
    f2 = np.fft.fft2(img2)  
    # Shift zero frequency component to the center
    f1_shifted = np.fft.fftshift(f1)
    f2_shifted = np.fft.fftshift(f2)  
    # Compute magnitude spectrum
    magnitude1 = np.abs(f1_shifted)
    magnitude2 = np.abs(f2_shifted)  
    # Compute the difference in magnitude spectra
    magnitude_diff = np.abs(magnitude1 - magnitude2)
    # Compute the difference index
    sum_magnitude1 = np.sum(magnitude1)
    sum_magnitude2 = np.sum(magnitude2)
    sum_magnitude_diff = np.sum(magnitude_diff)
    difference_index = sum_magnitude_diff / (sum_magnitude1 + sum_magnitude2 + 1e-6)  # Avoid division by zero
    return difference_index

# Combined All Metrics

In [ ]:
def Combined_Metrics(image1, image2,op):
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid")
    tsnr = TSNR(image1,image2)
    adif = Abs_Dif(image1,image2)
    epe = OF_EPE(image1,image2)
    ae = OF_AE(image1,image2)
    gray = Gray_Dif(image1,image2)
    ssim_value = TSSIM(image1,image2)
    mse_value = MSE(image1,image2)
    border_consistency_value = Border_Err(image1,image2)
    crci_value = CRC(image1,image2)
    ent = Entropy_Dif(image1,image2)
    # Normalize and combine the metrics into a single consistency index
    metrics = np.array([tsnr,adif, epe, ae, gray,ssim_value, mse_value, border_consistency_value, crci_value, ent])
    normalized_metrics = (metrics - np.min(metrics)) / (np.max(metrics) - np.min(metrics))
    combined_consistency_index = np.mean(normalized_metrics)
    #print(metrics)
    # Z-score normalization
    mean = np.mean(metrics)
    std = np.std(metrics)
    Z_metrics = (metrics - mean) / std
    
    if op=="norm":
        return combined_consistency_index
    if op=="mean":
        return np.mean(metrics)
    if op=="log":
        metrics[metrics <= 0] = 1e-10
        return np.sum(np.log(metrics))
    if op=="Z":
        return np.mean(Z_metrics)

# Mixed Metrics

In [ ]:
@numba.jit
def Mix_Metrics(image1, image2,image3,op="mean",M = [TSSIM,MSE],W=[1,1]):
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid")
    metrics = []
    for i in range(len(M)):
        #print(image1.shape,image2.shape)
        metrics.append(W[i]*M[i](image1,image2,image3))
    # Normalize and combine the metrics into a single consistency index
    metrics = np.array(metrics)
    normalized_metrics = (metrics - np.min(metrics)) / (np.max(metrics) - np.min(metrics))
    combined_consistency_index = np.mean(normalized_metrics)
    #print(metrics)
    # Z-score normalization
    mean = np.mean(metrics)
    std = np.std(metrics)
    Z_metrics = (metrics - mean) / std
    
    if op=="norm":
        return combined_consistency_index
    if op=="mean":
        return np.mean(metrics)
    if op=="log":
        metrics[metrics <= 0] = 1e-10
        return np.sum(np.log(metrics))
    if op=="Z":
        return np.mean(Z_metrics)

# Windowed Max Inconsistency

In [ ]:
def WMax_Inconsistency(img1, img2, img3, wsize=(3,3), step=(3,3), Func=Combined_Metrics, op="mean",M=[],Weights=[]):
    kw,kh = wsize
    H,W,_ = img1.shape
    sw,sh = step
    maxC = 0
    maxR = [0,0,kw,kh]
    # Calculate the output dimensions
    output_height = (H - kh) // sh + 1
    output_width = (W - kw) // sw + 1
    # Initialize the output array
    result = np.zeros((output_height, output_width))
    # Perform the convolution
    for i in range(0, output_height*sh, sh):
        #print('Row: ',i,'/',output_height*sh,end='\r')
        for j in range(0, output_width*sw, sw):
            # Extract the region of interest
            region1 = img1[i:i + kh, j:j + kw]
            region2 = img2[i:i + kh, j:j + kw]
            region3 = img3[i:i + kh, j:j + kw]
            # Perform element-wise multiplication and sum the result
            if Func==Combined_Metrics or Func==Mix_Metrics:
                if len(M)>0 and Func==Mix_Metrics:
                    result[i // sh, j // sw] = Func(region1, region2, region3,op,M,Weights)
                else:
                    result[i // sh, j // sw] = Func(region1, region2,region3,op)
            else:
                result[i // sh, j // sw] = Func(region1, region2,region3)
            if result[i//sh,j//sw]>=maxC:
                maxR = [i,i+kh,j,j+kw] 
                maxC = result[i//sh,j//sw]
            if (j+sw+kw)>W:
                break
        if (i+sh+kh)>H:
            break
    #print('')
    return result,maxR

# Video Consistency

In [ ]:
def Vid_consistency(F,Func=Combined_Metrics,op="mean",M=[],W=[]):
    C = []
    for i in range(len(F)-2):
        if Func==Combined_Metrics or Func==Mix_Metrics:
            if len(M)>0 and Func==Mix_Metrics:
                C.append(Func(F[i],F[i+1],F[i+2],op,M,W))
            else:
                C.append(Func(F[i],F[i+1],F[i+2],op))
        else:
            C.append(Func(F[i],F[i+1],F[i+2]))
    return np.mean(np.asarray(C)),C

# Windowed Video Consistency

In [ ]:
def Vid_consistency_W(F,wsize=(3,3),step=(3,3),Func=Combined_Metrics,op="mean",M=[],W=[]):
    C = []
    for i in range(len(F)-1):
        print('Frame: ',i+1,'/',len(F)-1,end='\r')
        Metrics,Region = WMax_Inconsistency(F[i],F[i+1],wsize,step,Func,op,W)
        C.append(np.mean(Metrics))
    return np.mean(np.asarray(C))

# Draw Window with Max Inconsistency

In [ ]:
#@jit
def DrawInconsistancy(F,wsize=(50,50),step=(50,50),Func=Combined_Metrics,op="mean",M=[],W=[]):
    L = 3
    DI = np.copy(F)
    for i in range(len(F)-2):
        print('Frame: ',i+1,'/',len(F)-1,end='\r')
        Metrics,Region = WMax_Inconsistency(F[i],F[i+1],F[i+2],wsize,step,Func,op,M,W)
        #print((Region[0],Region[2]),(Region[1],Region[3]))
        DI[i+1][Region[0]:Region[0]+L,Region[2]:Region[3]] = [0,255,0] 
        DI[i+1][Region[0]:Region[1],Region[2]:Region[2]+L] = [0,255,0] 
        DI[i+1][Region[1]:Region[1]+L,Region[2]:Region[3]] = [0,255,0] 
        DI[i+1][Region[0]:Region[1],Region[3]:Region[3]+L] = [0,255,0] 
        #DI[i+1] = cv2.putText(DI[i+1],str(np.mean(Metrics)),(10,20),cv2.FONT_HERSHEY_SIMPLEX,0.25,(0,0,255),1,cv2.LINE_AA) 
    return DI

# Draw Window with Max Inconsistency of Diffrent Window Sizes

In [ ]:
#@jit
def DrawInconsistancy1(F,Func=Combined_Metrics,op="mean",M=[],W=[]):
    L = 3
    DI = np.copy(F)
    for i in range(1,len(F)-1):
        print('Frame: ',i,'/',len(F)-1,end='\r')
        R = -1000
        Region = [0,0,0,0]
        for j in range(50,100,50):
            Metrics,Reg= WMax_Inconsistency(F[i-1],F[i],F[i+1],(j,j),(j//2,j//2),Func,op,M,W)
            if np.mean(Metrics)>R:
                R = np.mean(Metrics)
                Region = Reg
        #print((Region[0],Region[2]),(Region[1],Region[3]))
        
        DI[i+1][Region[0]:Region[0]+L,Region[2]:Region[3]] = [0,255,0] 
        DI[i+1][Region[0]:Region[1],Region[2]:Region[2]+L] = [0,255,0] 
        DI[i+1][Region[1]:Region[1]+L,Region[2]:Region[3]] = [0,255,0] 
        DI[i+1][Region[0]:Region[1],Region[3]:Region[3]+L] = [0,255,0] 
        #DI[i+1] = cv2.putText(DI[i+1],str(R),(10,20),cv2.FONT_HERSHEY_SIMPLEX,0.25,(0,0,255),1,cv2.LINE_AA) 
    return DI

In [ ]:
#@jit
def InconsistentRegion(F,Func=Combined_Metrics,op="mean",M=[],W=[]):
    L = 3
    h,w,_ = F[0].shape
    DI = [np.zeros((h,w)) for _ in F]
    for i in range(len(F)-2):
        #print('Frame: ',i+1,'/',len(F)-1,end='\r')
        R = -1000
        Region = [0,0,0,0]
        for j in range(20,100,20):
            Metrics,Reg= WMax_Inconsistency(F[i],F[i+1],F[i+2],(j,j),(j//2,j//2),Func,op,M,W)
            if np.mean(Metrics)>R:
                R = np.mean(Metrics)
                Region = Reg
        #print((Region[0],Region[2]),(Region[1],Region[3]))
        
        DI[i+1][Region[0]:Region[1],Region[2]:Region[3]] = 255
    return DI,R

# Torch to Numpy and Vice versa

In [ ]:
def Frame2Torch(Frame,Size=None,Normalize=False,Denormalize=False):
    if Size is not None:
        interpolation = cv2.INTER_CUBIC if Size[0]>Frame.shape[0] or Size[1]>Frame.shape[1] else cv2.INTER_AREA
        Frame = cv2.resize(Frame,Size,interpolation=interpolation)
    if Normalize:
        Frame = Frame.astype(float)/255.0
    if Denormalize:
        Frame = Frame.astype(float)*255.0
    return torch.tensor(Frame).permute(2, 0, 1).unsqueeze(0)

In [ ]:
def Frame2Numpy(Frame, Size=None, Normalize=False, Denormalize=False):
    if Frame.ndim == 2:  # If the tensor has shape (N, M)
        Frame = Frame.detach().cpu().numpy()
    elif Frame.ndim == 3:  # If the tensor has shape (C, H, W)
        Frame = Frame.squeeze(0).permute(1, 2, 0).detach().cpu().numpy()
    if Size is not None:
        interpolation = cv2.INTER_CUBIC if Size[0] > Frame.shape[0] or Size[1] > Frame.shape[1] else cv2.INTER_AREA
        Frame = cv2.resize(Frame, Size, interpolation=interpolation)
    if Normalize:
        Frame = Frame.astype(float) / 255.0
    if Denormalize:
        Frame = (Frame.astype(float) * 255.0).clip(0, 255).astype(np.uint8)
    return Frame

In [ ]:
def OF_tensor1(img1: torch.Tensor, img2: torch.Tensor, alpha: float = 1.0, iterations: int = 100, kernel_size: int = 3):
    import torch.nn.functional as Fun

    assert img1.shape == img2.shape, "Images must have the same shape"

    def rgb_to_grayscale(img):
        return 0.2989 * img[0, :, :] + 0.5870 * img[1, :, :] + 0.1140 * img[2, :, :]

    I1 = rgb_to_grayscale(img1)
    I2 = rgb_to_grayscale(img2)

    # Initialize optical flow vectors (u for x direction, v for y direction)
    u = torch.zeros_like(I1, requires_grad=True)
    v = torch.zeros_like(I1, requires_grad=True)
    
    # Define convolution kernels for gradients
    kernel_x = torch.tensor([[[[-1, 1], [-1, 1]]]], dtype=torch.float32)
    kernel_y = torch.tensor([[[[-1, -1], [1, 1]]]], dtype=torch.float32)

    # Compute gradients with padding that maintains the original image size
    Ix = Fun.conv2d(I1.unsqueeze(0).unsqueeze(0), kernel_x, padding=(1, 1)).squeeze(0).squeeze(0)
    Iy = Fun.conv2d(I1.unsqueeze(0).unsqueeze(0), kernel_y, padding=(1, 1)).squeeze(0).squeeze(0)
    It = I2 - I1  # Temporal gradient

    # Ensure all tensors have matching dimensions
    min_h = min(Ix.shape[-2], Iy.shape[-2], It.shape[-2], I1.shape[-2])
    min_w = min(Ix.shape[-1], Iy.shape[-1], It.shape[-1], I1.shape[-1])

    Ix = Ix[:min_h, :min_w]
    Iy = Iy[:min_h, :min_w]
    It = It[:min_h, :min_w]
    u = u[:min_h, :min_w]
    v = v[:min_h, :min_w]

    # Adjust padding for the specified kernel size
    padding = kernel_size // 2

    # Iteratively update the optical flow
    for _ in range(iterations):
        u_avg = Fun.avg_pool2d(u.unsqueeze(0).unsqueeze(0), kernel_size, stride=1, padding=padding).squeeze(0).squeeze(0)
        v_avg = Fun.avg_pool2d(v.unsqueeze(0).unsqueeze(0), kernel_size, stride=1, padding=padding).squeeze(0).squeeze(0)
        
        P = Ix * u_avg + Iy * v_avg + It
        D = alpha ** 2 + Ix ** 2 + Iy ** 2
        
        u = u_avg - (Ix * P) / D
        v = v_avg - (Iy * P) / D

    flow = torch.stack((u, v), dim=0)
    return flow


In [ ]:
def OF_tensor(img1: torch.Tensor, img2: torch.Tensor, alpha: float = 1.0, iterations: int = 100):
    assert img1.shape == img2.shape, "Images must have the same shape"

    def rgb_to_grayscale(img):
        return 0.2989 * img[0, :, :] + 0.5870 * img[1, :, :] + 0.1140 * img[2, :, :]

    I1 = rgb_to_grayscale(img1)
    I2 = rgb_to_grayscale(img2)

    # Initialize optical flow vectors (u for x direction, v for y direction)
    u = torch.zeros_like(I1, requires_grad=True)
    v = torch.zeros_like(I1, requires_grad=True)
    
    # Define convolution kernels for gradients
    kernel_x = torch.tensor([[[[-1, 1], [-1, 1]]]], dtype=torch.float32)
    kernel_y = torch.tensor([[[[-1, -1], [1, 1]]]], dtype=torch.float32)

    # Compute gradients with padding that maintains the original image size
    Ix = Fun.conv2d(I1.unsqueeze(0).unsqueeze(0), kernel_x, padding=(1, 1)).squeeze(0).squeeze(0)
    Iy = Fun.conv2d(I1.unsqueeze(0).unsqueeze(0), kernel_y, padding=(1, 1)).squeeze(0).squeeze(0)
    It = I2 - I1  # Temporal gradient

    # Ensure all tensors have matching dimensions
    min_h = min(Ix.shape[-2], Iy.shape[-2], It.shape[-2], I1.shape[-2])
    min_w = min(Ix.shape[-1], Iy.shape[-1], It.shape[-1], I1.shape[-1])

    Ix = Ix[:min_h, :min_w]
    Iy = Iy[:min_h, :min_w]
    It = It[:min_h, :min_w]
    u = u[:min_h, :min_w]
    v = v[:min_h, :min_w]

    # Iteratively update the optical flow
    for _ in range(iterations):
        u_avg = Fun.avg_pool2d(u.unsqueeze(0).unsqueeze(0), 3, stride=1, padding=1).squeeze(0).squeeze(0)
        v_avg = Fun.avg_pool2d(v.unsqueeze(0).unsqueeze(0), 3, stride=1, padding=1).squeeze(0).squeeze(0)
        
        P = Ix * u_avg + Iy * v_avg + It
        D = alpha ** 2 + Ix ** 2 + Iy ** 2
        
        u = u_avg - (Ix * P) / D
        v = v_avg - (Iy * P) / D

    flow = torch.stack((u, v), dim=0)
    return flow

In [ ]:
def draw_optical_flow1(img: torch.Tensor, flow: torch.Tensor, step: int = 10, scale: float = 1.0):
    # Convert the image to a NumPy array for OpenCV
    img_np = img.permute(1, 2, 0).cpu().numpy()  # (H, W, 3)
    img_np = (img_np * 255).astype(np.uint8)  # Scale to 0-255

    # Convert grayscale if needed
    if img_np.shape[2] == 1 or len(img_np.shape) == 2:
        img_np = cv2.cvtColor(img_np, cv2.COLOR_GRAY2BGR)

    # Extract flow components
    u = flow[0].cpu().detach().numpy()
    v = flow[1].cpu().detach().numpy()

    # Create a grid for drawing arrows
    H, W = u.shape
    y, x = np.mgrid[step // 2:H:step, step // 2:W:step].astype(np.int32)

    # Start with the base image
    display_img = img_np.copy()

    # Draw arrows
    for i, j in zip(x.flatten(), y.flatten()):
        dx = int(scale * u[j, i])
        dy = int(scale * v[j, i])
        pt1 = (i, j)
        pt2 = (i + dx, j + dy)
        color = (0, 255, 0)  # Green arrows
        thickness = 1
        cv2.arrowedLine(display_img, pt1, pt2, color, thickness, tipLength=0.3)

    # Display the result
    #cv2.imshow("Optical Flow", display_img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    return display_img

In [ ]:
def Edges_tensor(Frame):
    K = torch.tensor([[[[0, -1, 0], [-1, 0, 1], [0, 1, 0]]]*3]*3, dtype=torch.float)
    return Fun.conv2d(Frame,K,padding=1,stride=1).squeeze(0)

In [ ]:
def get_video_frames(path, N, size):
    # Find all video files in the specified path
    video_files = [f for f in os.listdir(path) if f.endswith(('.mp4', '.avi', '.mov', '.mkv'))]
    if not video_files:
        raise ValueError("No video files found in the specified path.")
    
    # Choose a random video file
    video_file = random.choice(video_files)
    video_path = os.path.join(path, video_file)
    
    # Initialize the video capture
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # If N is larger than the number of frames in the video, adjust it
    if N > total_frames:
        raise ValueError(f"The video has only {total_frames} frames, but {N} frames were requested.")
    
    # Select a random starting frame index such that we can capture N consecutive frames
    start_frame = random.randint(0, total_frames - N)
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    
    # Resize transformation
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(size),
        transforms.ToTensor()
    ])
    
    frames = []
    for _ in range(N):
        ret, frame = cap.read()
        if not ret:
            break
        # Convert the frame (BGR to RGB) and apply the resize transform
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_tensor = transform(frame_rgb)
        frames.append(frame_tensor[[2, 1, 0], :, :])
    
    cap.release()
    
    # Stack frames and reshape to the desired output shape
    frames_tensor = torch.stack(frames).unsqueeze(0)  # Shape (1, N, 3, H, W)
    
    return frames#_tensor

In [ ]:
def AddOneInc1(F,Op=None,x=None,y=None,xl=None,yl=None):
    # Convert to NumPy Array
    F = F.squeeze(0).permute(1, 2, 0).cpu().numpy() * 255.0
    # Copy of original array
    I = F.astype(np.uint8).copy()
    # Random location
    if x is None:
        x = random.randint(0, 2 * I.shape[1] // 3)
    if y is None:
        y = random.randint(0, 2 * I.shape[0] // 3)
    # Random size
    if xl is None:
        xl = random.randint(5, I.shape[1] // 6)
    if yl is None:
        yl = random.randint(5, I.shape[0] // 6)
    # Random option
    if Op is None:
        Op = random.randint(0, 20)
    #Change Color Range
    if Op == 0:
        I[x:x + xl, y:y + yl] = change_range_colors(
            I[x:x + xl, y:y + yl],
            (random.randint(30, 140), random.randint(30, 140), random.randint(30, 140)),
            (random.randint(150, 255), random.randint(150, 255), random.randint(150, 255))
        )
    #Color Rectangle
    elif Op == 1:
        R, G, B = random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)
        I[x:x + xl, y:y + yl] = change_range_colors(I[x:x + xl, y:y + yl], (R, G, B), (R, G, B))
    #Color Line
    elif Op == 2:
        Thick = random.randint(1, 10)
        l2 = random.randint(5, I.shape[1] // 15)
        I = cv2.line(I, (x, y), (x + xl, y + yl), (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)), Thick)
    #Black Line
    elif Op == 3:
        Thick = random.randint(1, 5)
        l2 = random.randint(5, I.shape[1] // 20)
        I = cv2.line(I, (x, y), (x + xl, y + yl), (0, 0, 0), Thick)
    #Add Noise
    elif Op == 4:
        I[x:x + xl, y:y + yl] = I[x:x + xl, y:y + yl] + np.random.randint(-10, 10, I[x:x + xl, y:y + yl].shape)
    #Convolution Random Kernel
    elif Op == 5:
        kernel = np.random.rand(3, 3)
        I[x:x + xl, y:y + yl] = cv2.filter2D(I[x:x + xl, y:y + yl], -1, kernel)
    # Deformation
    elif Op>5 and Op<12:
        I[x:x+xl,y:y+yl] = img_deformation(I[x:x+xl,y:y+yl],random.randint(10,80),random.randint(10,200))
    else:# Op == 7:
        I[x:x+xl,y:y+yl] = random_point_warp(I[x:x+xl,y:y+yl],random.randint(1,10))
    
    # Convert back to tensor
    I_tensor = torch.tensor(I, dtype=torch.float32).permute(2, 0, 1) / 255.0
    return I_tensor

In [ ]:
def AddOneInc2(Bin,F,Op=None,x=None,y=None,xl=None,yl=None):
    Bin = Frame2Numpy(Bin)
    # Convert to NumPy Array
    F = F.squeeze(0).permute(1, 2, 0).cpu().numpy() * 255.0
    # Copy of original array
    I = F.astype(np.uint8).copy()
    # Random location
    if x is None:
        x = random.randint(0, 2 * I.shape[1] // 3)
    if y is None:
        y = random.randint(0, 2 * I.shape[0] // 3)
    # Random size
    if xl is None:
        xl = random.randint(5, I.shape[1] // 6)
    if yl is None:
        yl = random.randint(5, I.shape[0] // 6)
    # Random option
    if Op is None:
        Op = random.randint(0, 12)
    #Change Color Range
    if Op == 0:
        I[x:x + xl, y:y + yl] = change_range_colors(
            I[x:x + xl, y:y + yl],
            (random.randint(30, 140), random.randint(30, 140), random.randint(30, 140)),
            (random.randint(150, 255), random.randint(150, 255), random.randint(150, 255))
        )
        Bin[x:x+xl,y:y+yl] = 1.0
    #Color Rectangle
    elif Op == 1:
        R, G, B = random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)
        I[x:x + xl, y:y + yl] = change_range_colors(I[x:x + xl, y:y + yl], (R, G, B), (R, G, B))
        Bin[x:x+xl,y:y+yl] = 1.0
    #Color Line
    elif Op == 2:
        Thick = random.randint(1, 11)
        if random.random()>0.5:
            yl = -yl
        I = cv2.line(I, (x, y), (x + xl, y + yl), (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)), Thick)
        Bin = cv2.line(Bin,(x,y),(x+xl,y+yl),(1,1,1),Thick)
    #Black Line
    elif Op == 3:
        Thick = random.randint(1, 5)
        if random.random()>0.5:
            yl = -yl
        I = cv2.line(I, (x, y), (x + xl, y + yl), (0, 0, 0), Thick)
        Bin = cv2.line(Bin,(x,y),(x+xl,y+yl),(1,1,1),Thick)
    #Add Noise
    elif Op == 4:
        I[x:x + xl, y:y + yl] = I[x:x + xl, y:y + yl] + np.random.randint(-10, 10, I[x:x + xl, y:y + yl].shape)
        Bin[x:x + xl, y:y + yl] = 1.0
    #Convolution Random Kernel
    elif Op == 5:
        kernel = np.random.rand(3, 3)
        I[x:x + xl, y:y + yl] = cv2.filter2D(I[x:x + xl, y:y + yl], -1, kernel)
        Bin[x:x + xl, y:y + yl] = 1.0
    # Deformation
    elif Op==6:
        I[x:x+xl,y:y+yl] = img_deformation(I[x:x+xl,y:y+yl],random.randint(10,80),random.randint(10,200))
        Bin[x:x + xl, y:y + yl] = 1.0
    #Warp
    elif Op==7:
        I[x:x+xl,y:y+yl] = random_point_warp(I[x:x+xl,y:y+yl],random.randint(1,10))
        Bin[x:x + xl, y:y + yl] = 1.0
    #Curved line
    elif Op==8:
        I,Bin = draw_random_curve(I,Bin,x,y,x+xl,y+yl)
    #Polygon
    elif Op==9:
        I,B = draw_random_polygon(I, Bin, x,y,x+xl,y+yl)
    #Curved Polygon
    elif Op==10:
        I,Bin = draw_random_curve_fig(I, Bin,x,y,x+xl,y+yl)
    #Change Lightning
    else:
        I,Bin = light_noise(I,Bin,x,x+xl,y,y+yl)
    # Convert back to tensor
    I_tensor = torch.tensor(I, dtype=torch.float32).permute(2, 0, 1) / 255.0
    Bin = torch.tensor(Bin,dtype=torch.float32)
    return I_tensor,Bin

In [ ]:
def display_frequency_image(freq_tensor):
    # Ensure the tensor is on the CPU and convert it to a NumPy array
    freq_numpy = freq_tensor.detach().cpu().numpy()
    
    # Handle cases where the tensor has extra dimensions
    if freq_numpy.ndim == 3 and freq_numpy.shape[0] in [1, 3]:  # (C, H, W) format
        freq_numpy = freq_numpy.transpose(1, 2, 0)  # Convert to (H, W, C) for OpenCV
    elif freq_numpy.ndim == 3:  # If it's a single-channel 3D tensor
        freq_numpy = freq_numpy[0]  # Take the first channel

    # Take the magnitude if it contains complex values
    if np.iscomplexobj(freq_numpy):
        freq_numpy = np.abs(freq_numpy)
    
    # Apply a logarithmic transformation for better visibility
    freq_numpy = np.log(1 + freq_numpy)  # Adding 1 to avoid log(0)

    # Normalize to the range [0, 255] for display
    freq_normalized = cv2.normalize(freq_numpy, None, 0, 255, cv2.NORM_MINMAX)
    freq_normalized = freq_normalized.astype(np.uint8)
    
    # Display the frequency image
    #cv2.imshow('Frequency Image', freq_normalized)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    
    return freq_normalized

In [ ]:
def create_gaussian_window(window_size, channels, sigma):
    # Create a 1D Gaussian kernel
    x = torch.arange(window_size, dtype=torch.float32) - (window_size - 1) / 2
    gauss = torch.exp(-(x**2) / (2 * sigma**2))
    gauss /= gauss.sum()  # Normalize the 1D kernel to ensure the sum is 1

    # Create a 2D Gaussian kernel by taking the outer product
    gauss_2d = gauss[:, None] * gauss[None, :]

    # Reshape the kernel to be compatible with conv2d
    kernel = gauss_2d.expand(channels, 1, window_size, window_size)

    return kernel

In [ ]:
def TSSIM_tensor(img1, img2, window_size=11, sigma=1.5, C1=0.01**2, C2=0.03**2):

    channels = img1.size(1)
    gaussian_window = create_gaussian_window(window_size, channels, sigma).to(img1.device)

    # Compute local means using Gaussian filter
    mu1 = Fun.conv2d(img1, gaussian_window, padding=window_size // 2, groups=channels)
    mu2 = Fun.conv2d(img2, gaussian_window, padding=window_size // 2, groups=channels)

    # Compute local variances and covariance
    mu1_sq = mu1 ** 2
    mu2_sq = mu2 ** 2
    mu1_mu2 = mu1 * mu2

    sigma1_sq = Fun.conv2d(img1 * img1, gaussian_window, padding=window_size // 2, groups=channels) - mu1_sq
    sigma2_sq = Fun.conv2d(img2 * img2, gaussian_window, padding=window_size // 2, groups=channels) - mu2_sq
    sigma12 = Fun.conv2d(img1 * img2, gaussian_window, padding=window_size // 2, groups=channels) - mu1_mu2

    # Compute SSIM map
    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / (
        (mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2)
    )

    mean_ssim = ssim_map.mean()  # Compute the mean SSIM value

    return mean_ssim, ssim_map  # Return both mean SSIM and SSIM map

In [1]:
#Draw Optical Flow
def draw_OF(image, flow, step=16, scale=1.0):
    output_image = image.copy()
    h, w = flow.shape[:2]
    for y in range(0, h, step):
        for x in range(0, w, step):
            fx, fy = flow[y, x]
            end_x = int((x + fx*scale))
            end_y = int((y + fy*scale))
            cv2.arrowedLine(output_image, (x, y), (end_x, end_y), (0, 255, 0), 1, tipLength=0.3)
    return output_image

In [1]:
def warp_OF(image, flow):
    h, w = flow.shape[:2]
    y, x = np.meshgrid(np.arange(h), np.arange(w), indexing='ij')
    new_x = (x + flow[..., 0])
    new_y = (y + flow[..., 1])
    warped_image = cv2.remap(image, new_x, new_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
    return warped_image

In [ ]:
def draw_bezier_curve(image,Bin, p0, p1, p2, p3, color, thickness=2):
    curve = []
    for t in np.linspace(0, 1, 100):  # Generate points along the Bézier curve
        x = (1 - t)**3 * p0[0] + 3 * (1 - t)**2 * t * p1[0] + 3 * (1 - t) * t**2 * p2[0] + t**3 * p3[0]
        y = (1 - t)**3 * p0[1] + 3 * (1 - t)**2 * t * p1[1] + 3 * (1 - t) * t**2 * p2[1] + t**3 * p3[1]
        curve.append((int(x), int(y)))
    for i in range(len(curve) - 1):
        cv2.line(image, curve[i], curve[i + 1], color, thickness)
        cv2.line(Bin,curve[i],curve[i+1],(1,1,1),thickness)

def draw_random_curve(image,Bin,x,y,xl,yl):
    p0 = (random.randint(x, xl), random.randint(y, yl))
    p1 = (random.randint(x, xl), random.randint(y, yl))
    p2 = (random.randint(x, xl), random.randint(y, yl))
    p3 = (random.randint(x, xl), random.randint(y, yl))
    color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    thickness = random.randint(1, 3)
    draw_bezier_curve(image,Bin, p0, p1, p2, p3, color, thickness)
    return image,Bin

In [ ]:
def draw_random_polygon(img, binary_img, x,y,xl,yl):
    # Generate a random number of points (between 3 and 8)
    num_points = random.randint(3, 8)

    # Generate random points within the image dimensions
    points = np.array([(random.randint(x, xl - 1), random.randint(y, yl - 1)) for _ in range(num_points)])

    # Randomly decide whether to sort the points to make it convex
    if random.choice([True, False]):
        hull = cv2.convexHull(points)  # Ensure convex shape
    else:
        hull = points  # Keep it concave

    hull = hull.reshape((-1, 1, 2))  # Ensure proper shape for OpenCV

    # Generate a random color
    color = [random.randint(0, 255) for _ in range(3)]

    # Draw the polygon on the image
    cv2.fillPoly(img, [hull], color)

    # Draw the polygon as 1s on the binary image
    cv2.fillPoly(binary_img, [hull], 1)

    return img, binary_img

In [ ]:
import cv2
import numpy as np
import random
from scipy.interpolate import splprep, splev

def draw_random_curve_fig(img, binary_img, x, y, xl, yl):
    # Ensure bounding box is valid
    if xl <= x or yl <= y:
        raise ValueError("Invalid bounding box: Ensure xl > x and yl > y.")

    # Generate 5-10 random control points
    num_points = random.randint(5, 10)
    control_points = np.array([(random.randint(x, xl), random.randint(y, yl)) for _ in range(num_points)], dtype=np.float32)

    # Remove duplicate points (splprep requires unique points)
    control_points = np.unique(control_points, axis=0)
    
    # Ensure we have enough points (at least 4 for cubic B-spline)
    if len(control_points) < 4:
        return img, binary_img  # Skip drawing if not enough points

    # Close the curve by repeating the first point
    control_points = np.vstack([control_points, control_points[0]])

    # Fit a smooth B-spline curve
    tck, u = splprep(control_points.T, s=0.5, per=True)
    u_new = np.linspace(0, 1, num_points * 10)  # Smooth interpolation
    curve = np.array(splev(u_new, tck)).T  # Generate curve points

    # Convert to integer format for OpenCV
    curve = np.round(curve).astype(np.int32).reshape(-1, 1, 2)

    # Generate a random color
    color = [random.randint(0, 255) for _ in range(3)]

    # Draw the filled curved shape on the image
    cv2.fillPoly(img, [curve], color)

    # Draw the shape in the binary image (fill with 255 for visibility)
    cv2.fillPoly(binary_img, [curve], 1)

    return img, binary_img


In [ ]:
def light_noise(image,Bin,x,xl,y,yl):
    image[y:yl, x:xl] = image[y:yl, x:xl]*random.gauss(0.5,1/6)
    Bin[y:yl,x:xl] = 1.0
    return image,Bin

In [ ]:
def Optical_flow_tensor(frame0, frame1):
    if frame0.max() <= 1:
        frame0 = frame0 * 255
        frame1 = frame1 * 255

    old_frame = Frame2Numpy(frame0)
    frame = Frame2Numpy(frame1)
    H, W = old_frame.shape[:2]

    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Compute dense optical flow using Farneback
    flow = cv2.calcOpticalFlowFarneback(old_gray, frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Convert flow to PyTorch tensor
    flow_tensor = torch.tensor(flow, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)  # (1, 2, H, W)

    return -flow_tensor

In [ ]:
def warp_image_tensor(image: torch.Tensor, flow: torch.Tensor) -> torch.Tensor:
    
    B, C, H, W = image.shape

    # Generate a mesh grid of coordinates (height, width)
    grid_y, grid_x = torch.meshgrid(torch.arange(H, device=image.device), 
                                    torch.arange(W, device=image.device), indexing='ij')
    grid = torch.stack((grid_x, grid_y), dim=-1).float()  # shape: (H, W, 2)

    # Normalize the grid to the range [-1, 1]
    grid = 2.0 * grid / torch.tensor([W - 1, H - 1], device=image.device) - 1.0  # scale to [-1, 1]
    grid = grid.view(1, H, W, 2).repeat(B, 1, 1, 1)

    flow = flow.permute(0, 2, 3, 1)  # (B, 2, H, W) -> (B, H, W, 2)

    # Normalize optical flow to grid scale
    scale = torch.tensor([(W - 1) / 2, (H - 1) / 2], device=image.device)
    flow = flow / scale  # Convert pixel displacement to [-1,1] range

    grid = grid.clone() + flow 

    # Sample the image using the new grid
    warped_image = Fun.grid_sample(image, grid, mode='bilinear', padding_mode='zeros', align_corners=True)

    return warped_image

In [ ]:
def Optical_flow_tensor1(frame0, frame1):
    if frame0.max() <= 1:
        frame0 = frame0 * 255
        frame1 = frame1 * 255
    
    frame0 = frame0.float()
    frame1 = frame1.float()
    
    # Convert to grayscale
    frame0_gray = 0.299 * frame0[:, 0, :, :] + 0.587 * frame0[:, 1, :, :] + 0.114 * frame0[:, 2, :, :]
    frame1_gray = 0.299 * frame1[:, 0, :, :] + 0.587 * frame1[:, 1, :, :] + 0.114 * frame1[:, 2, :, :]
    
    # Compute pyramidal downsampling like Farneback
    frame0_down = Fun.avg_pool2d(frame0_gray.unsqueeze(1), kernel_size=2, stride=2)
    frame1_down = Fun.avg_pool2d(frame1_gray.unsqueeze(1), kernel_size=2, stride=2)
    
    # Compute image gradients (smooth approximation)
    sobel_x = torch.tensor([[[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]], dtype=torch.float32, device=frame0.device).view(1, 1, 3, 3)
    sobel_y = torch.tensor([[[-1, -2, -1], [0, 0, 0], [1, 2, 1]]], dtype=torch.float32, device=frame0.device).view(1, 1, 3, 3)
    
    Ix = Fun.conv2d(frame0_down, sobel_x, padding=1)
    Iy = Fun.conv2d(frame0_down, sobel_y, padding=1)
    It = (frame1_down - frame0_down)
    
    # Compute flow estimation using iterative refinement
    eps = 1e-6
    flow_x = -It * Ix / (Ix**2 + Iy**2 + eps)
    flow_y = -It * Iy / (Ix**2 + Iy**2 + eps)
    
    # Upsample flow back to original resolution
    flow_x = Fun.interpolate(flow_x, size=frame0_gray.shape[-2:], mode='bilinear', align_corners=False)
    flow_y = Fun.interpolate(flow_y, size=frame0_gray.shape[-2:], mode='bilinear', align_corners=False)
    
    flow = torch.cat([flow_x, flow_y], dim=1)  # Shape (B, 2, H, W)
    return -flow

In [ ]:
def compute_variational_optical_flow(img1, img2, alpha=1.0, num_iters=10):
    # Convert RGB to grayscale using simple averaging
    img1_gray = img1.mean(dim=0, keepdim=True)
    img2_gray = img2.mean(dim=0, keepdim=True)
    
    # Compute gradients
    sobel_x = torch.tensor([[1, 0, -1], [2, 0, -2], [1, 0, -1]], dtype=torch.float32).view(1, 1, 3, 3)
    sobel_y = sobel_x.transpose(2, 3)
    
    I_x = Fun.conv2d(img1_gray.unsqueeze(0), sobel_x, padding=1)[0]
    I_y = Fun.conv2d(img1_gray.unsqueeze(0), sobel_y, padding=1)[0]
    I_t = img2_gray - img1_gray  # Temporal derivative

    # Initialize flow (zero displacement)
    u = torch.zeros_like(img1_gray)
    v = torch.zeros_like(img1_gray)

    # Smoothness kernel
    avg_kernel = torch.ones(1, 1, 3, 3) / 9.0

    for _ in range(num_iters):
        # Compute average flow
        u_avg = Fun.conv2d(u.unsqueeze(0), avg_kernel, padding=1)[0]
        v_avg = Fun.conv2d(v.unsqueeze(0), avg_kernel, padding=1)[0]

        # Solve for flow using iterative updates
        flow_denom = alpha + I_x**2 + I_y**2
        u = u_avg - (I_x * (I_x * u_avg + I_y * v_avg + I_t)) / flow_denom
        v = v_avg - (I_y * (I_x * u_avg + I_y * v_avg + I_t)) / flow_denom

    return torch.cat([u, v], dim=0)  # (2, H, W)


In [ ]:
def pyramid_optical_flow(img1, img2, num_levels=3, alpha=0.1, num_iters=10):
    flow = torch.zeros(2, img1.shape[1], img1.shape[2], device=img1.device)
    
    for level in range(num_levels, -1, -1):
        scale = 2**level
        H, W = img1.shape[1] // scale, img1.shape[2] // scale
        
        img1_resized = Fun.interpolate(img1.unsqueeze(0), size=(H, W), mode="bilinear", align_corners=False).squeeze(0)
        img2_resized = Fun.interpolate(img2.unsqueeze(0), size=(H, W), mode="bilinear", align_corners=False).squeeze(0)

        flow_resized = Fun.interpolate(flow.unsqueeze(0), size=(H, W), mode="bilinear", align_corners=False).squeeze(0)
        
        # Compute flow at this level (using variational or another method)
        flow_update = compute_variational_optical_flow(img1_resized, img2_resized, alpha, num_iters)
        
        flow = flow_resized + flow_update  # Refine estimate
        
    return flow*2**num_levels

In [ ]:
def differentiable_segmentation(image, num_clusters=3, sigma=0.1):
    C, H, W = image.shape
    image_flat = image.view(C, -1).T  # Reshape to (H*W, C)
    
    # Initialize cluster centers equally distributed within image range
    min_val, max_val = image_flat.min(dim=0)[0], image_flat.max(dim=0)[0]
    cluster_centers = torch.linspace(0, 1, num_clusters).view(-1, 1) * (max_val - min_val) + min_val
    cluster_centers = cluster_centers.to(image.device).requires_grad_(True)
    
    # Compute similarity between pixels and cluster centers
    distances = torch.cdist(image_flat, cluster_centers)  # (H*W, num_clusters)
    
    # Apply soft assignment using a Gaussian kernel
    soft_assignments = Fun.softmax(-distances / sigma, dim=-1)  # (H*W, num_clusters)
    
    # Reshape back to (num_clusters, H, W)
    segmentation = soft_assignments.T.view(num_clusters, H, W)

    # Generate equally spaced colors in RGB space
    cluster_colors = torch.linspace(0, 1, num_clusters).unsqueeze(1).expand(-1, 3)  # (num_clusters, 3)
    
    # Make it a learnable tensor
    cluster_colors = torch.nn.Parameter(cluster_colors.to(image.device))

    # Compute RGB image as a weighted sum of cluster colors
    segmented_rgb = torch.einsum('chw,cn->nhw', segmentation, cluster_colors)  # Shape: (3, H, W)
    
    return segmented_rgb

In [ ]:
def differentiable_segmentationRGB(image, num_clusters=3, sigma=0.1):
    C, H, W = image.shape
    image_flat = image.view(C, -1).T  # Reshape to (H*W, C)
    
    # Initialize cluster centers in RGB space
    min_val, max_val = image_flat.min(dim=0)[0], image_flat.max(dim=0)[0]
    values = torch.linspace(0, 1, num_clusters)

    # Create all combinations by repeating the values for each dimension
    cluster_centers = torch.cartesian_prod(values, values, values).float()

    num_clusters = cluster_centers.shape[0]
    
    # Compute similarity between pixels and cluster centers
    distances = torch.cdist(image_flat, cluster_centers)  # (H*W, num_clusters)
    
    # Apply soft assignment using a Gaussian kernel
    soft_assignments = Fun.softmax(-distances / sigma, dim=-1)  # (H*W, num_clusters)
    
    # Reshape back to (num_clusters, H, W)
    segmentation = soft_assignments.T.view(num_clusters, H, W)
    
    # Compute RGB image as a weighted sum of cluster colors
    segmented_rgb = torch.einsum('chw,cn->nhw', segmentation, cluster_centers)  # Shape: (3, H, W)

    return segmented_rgb